In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
import math
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score#R square
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from boruta import BorutaPy
from ngboost.ngboost import NGBoost
from ngboost.learners import default_tree_learner
from ngboost.distns import Normal
from ngboost.scores import MLE
from ngboost.scores import CRPS
from ngboost import NGBRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from mealpy.utils.visualize import *
from mealpy.bio_based import BBO, EOA, IWO, SBO, SMA, TPO, VCS, WHO
from mealpy.evolutionary_based import CRO, DE, EP, ES, FPA, GA, MA
from mealpy.human_based import BRO, BSO, CA, CHIO, FBIO, GSKA, ICA, LCO, QSA, SARO, SSDO, TLO
from mealpy.math_based import AOA, CGO, GBO, HC, SCA
from mealpy.music_based import HS
from mealpy.physics_based import ArchOA, ASO, EFO, EO, HGSO, MVO, NRO, SA, TWO, WDO
from mealpy.system_based import AEO, GCO, WCA
from mealpy.swarm_based import ABC, ACOR, ALO, AO, BA, BeesA, BES, BFO, BSA, COA, CSA, CSO, DO, EHO, FA, FFA, FOA, GOA, GWO, HGS
from mealpy.swarm_based import HHO, JA, MFO, MRFO, MSA, NMRA, PFA, PSO, SFO, SHO, SLO, SRSR, SSA, SSO, SSpiderA, SSpiderO, WOA
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
import warnings
from lightgbm import LGBMRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import catboost as cat
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')
from catboost import CatBoostRegressor
import catboost as cat
from sklearn.tree import DecisionTreeRegressor
from ngboost.scores import LogScore, CRPScore
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor

# Data Preprocessing

In [ ]:
data = pd.read_csv('MLCORE_2.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
df=data

In [ ]:
df.info()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager

# Load the Times New Roman font
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12

# From CSV file
def read_data_from_csv(csv_file):
    data = pd.read_csv(csv_file)
    return data

# Plot histograms in subplots
def plot_histograms_in_subplots(data):
    num_rows = 4
    num_cols = 4

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 12))
    variables = data.columns.tolist()

    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            if index < len(variables):
                variable_name = variables[index]
                axes[i, j].hist(data[variable_name],  bins=20, alpha=0.7, color='deepskyblue',edgecolor='black')
                axes[i, j].tick_params(axis='both', labelsize=12)
                # Move the variable name below the subplot
                axes[i, j].text(0.5, -0.2, variable_name, va='center', ha='center', transform=axes[i, j].transAxes, fontproperties=font_prop)
            else:
                axes[i, j].axis('off')

    plt.tight_layout()
    plt.savefig('datahist.jpg',dpi=1500, bbox_inches='tight')
    plt.show()

if __name__ == "__main__":
    csv_file_path = "MLCORE_3.csv"   # Replace with your CSV file path
    data = read_data_from_csv(csv_file_path)
    plot_histograms_in_subplots(data)


# Data Split

In [ ]:
from sklearn.model_selection import train_test_split
dataframe = df
dataframe['PERM1'] = pd.cut(dataframe['PERM'],
                    bins=[-2.,-1.,0.,1.,2.,3.],
                    labels=[1,2,3,4,5])
dataframe['PERM1'].hist()
print(pd.value_counts(dataframe['PERM1'])) 

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=10, test_size=0.3,  random_state=42) 
for train_index, test_index in split.split(dataframe, dataframe['PERM1']):
    strat_train_set = dataframe.iloc[train_index]      
    strat_test_set = dataframe.iloc[test_index]      

In [ ]:
train_set = strat_train_set.copy()
test_set = strat_test_set.copy()
train_set.head()

In [ ]:
#train_set1 = train_set.drop(['DEPTH','BIT','CAL'],axis=1)
#test_set1 = test_set.drop(['DEPTH','BIT','CAL'],axis=1)
train_set1 = train_set
test_set1 = test_set

In [ ]:
train_set1.head()

In [ ]:
test_set1.head()

In [ ]:
train_set11 = train_set1.values
train_set11 = train_set11.astype('float32')
test_set11 = test_set1.values
test_set11 = test_set11.astype('float32')

In [ ]:
#train_set = train_set.values
X_train = train_set11[:,0:16]
#y_por_train = train_set11[:,16]
y_perm_train = train_set11[:,16]
print(X_train.shape)
#print(y_por_train.shape)
print(y_perm_train.shape)

In [ ]:
#test_set = test_set.va0ues
X_test = test_set11[:,0:16]
#y_por_test = test_set11[:,16]
y_perm_test = test_set11[:,16]
print(X_test.shape)
#print(y_por_test.shape)
print(y_perm_test.shape)

In [ ]:
X_train_ys = pd.DataFrame(X_train,columns=['DEPTH','SP', 'GR','BIT','CAL', 'CNL', 'DEN', 'DTC', 'DTS','P16H', 'P28H', 'P40H', 'A40H', 'SH','KJ','POR_LOG'])
X_train_ys.head()

In [ ]:
y_train_ys = pd.DataFrame(y_perm_train,columns=['PERM'])
y_train_ys.head()

In [ ]:
X_test_ys = pd.DataFrame(X_test,columns=['DEPTH','SP', 'GR','BIT','CAL', 'CNL', 'DEN', 'DTC', 'DTS','P16H', 'P28H', 'P40H', 'A40H', 'SH','KJ','POR_LOG'])
X_test_ys.head()

In [ ]:
y_test_ys = pd.DataFrame(y_perm_test,columns=['PERM'])
y_test_ys.head()

# Feature Selection

# 1. Pearson

In [ ]:
corr_matrix =  train_set1.corr()
corr_matrix

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1, 1, figsize=(18, 10))
plt.tick_params(labelsize=15)
plt.yticks(fontproperties='Times New Roman', size=15,weight='bold')#
plt.xticks(fontproperties='Times New Roman', size=15,weight='bold')
sns.heatmap(data=corr_matrix, 
            annot=True,
            annot_kws={"size":12,"font":'Times New Roman'},
            square=True,
            cmap='vlag',
            fmt='.2f', 
            linewidths=1,
            linecolor='w',
            ax=ax)
plt.savefig('Pearson correlation matrix.svg', dpi=1500, bbox_inches='tight')

In [ ]:
X_train_pearson = X_train_ys.drop(['SP','BIT','CAL','P40H','A40H'],axis=1)
X_train_pearson .head()

In [ ]:
X_test_pearson = X_test_ys.drop(['SP','BIT','CAL','P40H','A40H'],axis=1)
X_test_pearson.head()

# 2.RFECV

# RFECV

In [ ]:
rfe_xg = xgb.XGBRegressor(random_state = 42)
rfecv_xg = RFECV(estimator=rfe_xg, step=1,min_features_to_select=1, cv=5, scoring='r2',n_jobs=-1,importance_getter='auto')
rfecv_xg.fit(X_train_ys,y_train_ys)
print('Optimal number of features: {}'.format(rfecv_xg.n_features_))

In [ ]:
# Plot number of features VS. cross-validation scores
figsize = 6,4
figure, ax = plt.subplots(figsize=figsize)
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.xlabel("Number of features",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("R\u00b2",fontproperties="Times New Roman",fontsize=20)
plt.plot(range(1, len(rfecv_xg.grid_scores_) + 1), rfecv_xg.grid_scores_, linewidth=3)
plt.tight_layout() 
plt.savefig('RFECV-XGBoost.jpg',dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
rfecv_xg.grid_scores_

In [ ]:
print(np.where(rfecv_xg.support_ == False)[0])
X_train_hbdf1 = pd.DataFrame(X_train_ys)
X_train_hbdf1.drop(X_train_hbdf1.columns[np.where(rfecv_xg.support_ == False)[0]], axis=1, inplace=True)

In [ ]:
rfecv_xg.estimator_.feature_importances_

In [ ]:
dset = pd.DataFrame()
dset['attr'] = X_train_hbdf1.columns
dset['importance'] = rfecv_xg.estimator_.feature_importances_
dset = dset.sort_values(by='importance', ascending=False)

In [ ]:
dset

In [ ]:
figsize = 6,4
figure, ax = plt.subplots(figsize=figsize)
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.barh(y=dset['attr'], width=dset['importance'], color='b')
plt.xlabel('Feature importance',fontproperties="Times New Roman",fontsize=20, labelpad=20)
plt.savefig('RFECV_XGBOOST-Feature importances.jpg',dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
rfe_lg = lgb.LGBMRegressor(random_state = 42)
rfecv_lg = RFECV(estimator=rfe_lg, step=1, cv=5, scoring='r2')
rfecv_lg.fit(X_train_ys,y_train_ys)
print('Optimal number of features: {}'.format(rfecv_lg.n_features_))

In [ ]:
# Plot number of features VS. cross-validation scores
figsize = 6,4
figure, ax = plt.subplots(figsize=figsize)
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.xlabel("Number of features",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("R\u00b2",fontproperties="Times New Roman",fontsize=20)
plt.plot(range(1, len(rfecv_lg.grid_scores_) + 1), rfecv_lg.grid_scores_, linewidth=3)
plt.savefig('RFECV-LightGBM.jpg',dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
rfecv_lg.grid_scores_

In [ ]:
print(np.where(rfecv_lg.support_ == False)[0])
X_train_hbdf1 = pd.DataFrame(X_train_ys)
X_train_hbdf1.drop(X_train_hbdf1.columns[np.where(rfecv_lg.support_ == False)[0]], axis=1, inplace=True)

In [ ]:
rfecv_lg.estimator_.feature_importances_

In [ ]:
dset = pd.DataFrame()
dset['attr'] = X_train_hbdf1.columns
dset['importance'] = rfecv_lg.estimator_.feature_importances_
dset = dset.sort_values(by='importance', ascending=False)

In [ ]:
dset

In [ ]:
figsize = 6,4
figure, ax = plt.subplots(figsize=figsize)
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.barh(y=dset['attr'], width=dset['importance'], color='b')
plt.xlabel('Feature importance',fontproperties="Times New Roman",fontsize=20, labelpad=20)
plt.savefig('RFECV_LightGBM-Feature importances.jpg',dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
rfe_rf =RandomForestRegressor(
        n_estimators = 500,  
        max_depth = 10,     
        random_state = 42)
rfecv_rf = RFECV(estimator=rfe_rf, step=1, cv=5, scoring='r2')
rfecv_rf.fit(X_train_ys,y_train_ys)
print('Optimal number of features: {}'.format(rfecv_rf.n_features_))

In [ ]:
figsize = 6,4
figure, ax = plt.subplots(figsize=figsize)
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False  
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.xlabel("Number of features",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("R\u00b2",fontproperties="Times New Roman",fontsize=20)
plt.plot(range(1, len(rfecv_rf.grid_scores_) + 1), rfecv_rf.grid_scores_, linewidth=3)
plt.savefig('RFECV-RF.jpg',dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
rfecv_rf.grid_scores_

In [ ]:
print(np.where(rfecv_rf.support_ == False)[0])
X_train_hbdf1 = pd.DataFrame(X_train_ys)
X_train_hbdf1.drop(X_train_hbdf1.columns[np.where(rfecv_rf.support_ == False)[0]], axis=1, inplace=True)

In [ ]:
rfecv_xg.estimator_.feature_importances_

In [ ]:
dset = pd.DataFrame()
dset['attr'] = X_train_hbdf1.columns
dset['importance'] = rfecv_rf.estimator_.feature_importances_
dset = dset.sort_values(by='importance', ascending=False)

In [ ]:
dset

In [ ]:
figsize = 6,4
figure, ax = plt.subplots(figsize=figsize)
plt.tick_params(labelsize=15)
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.barh(y=dset['attr'], width=dset['importance'], color='b')
plt.xlabel('Feature importance',fontproperties="Times New Roman",fontsize=20, labelpad=20)
plt.savefig('RFECV_RF-Feature importances.jpg',dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
rfe_cat = cat.CatBoostRegressor(random_state = 42)
rfecv_cat = RFECV(estimator=rfe_cat, step=1, cv=5, scoring='r2')
rfecv_cat.fit(X_train_ys,y_train_ys)
print('Optimal number of features: {}'.format(rfecv_cat.n_features_))

In [ ]:
# Plot number of features VS. cross-validation scores
figsize = 6,4
figure, ax = plt.subplots(figsize=figsize)
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False  
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.xlabel("Number of features",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("R\u00b2",fontproperties="Times New Roman",fontsize=20)
plt.plot(range(1, len(rfecv_cat.grid_scores_) + 1), rfecv_cat.grid_scores_, linewidth=3)
plt.savefig('RFECV-CATBoost.jpg',dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
rfecv_cat.grid_scores_

In [ ]:
print(np.where(rfecv_cat.support_ == False)[0])
X_train_hbdf1 = pd.DataFrame(X_train_ys)
X_train_hbdf1.drop(X_train_hbdf1.columns[np.where(rfecv_cat.support_ == False)[0]], axis=1, inplace=True)

In [ ]:
rfecv_cat.estimator_.feature_importances_

In [ ]:
dset = pd.DataFrame()
dset['attr'] = X_train_hbdf1.columns
dset['importance'] = rfecv_cat.estimator_.feature_importances_
dset = dset.sort_values(by='importance', ascending=False)

In [ ]:
dset

In [ ]:
figsize = 6,4
figure, ax = plt.subplots(figsize=figsize)
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.barh(y=dset['attr'], width=dset['importance'], color='b')
plt.xlabel('Feature importance',fontproperties="Times New Roman",fontsize=20, labelpad=20)
plt.savefig('RFECV_CATBoost-Feature importances.jpg',dpi=1500, bbox_inches='tight')
plt.show()

# Boruta 

In [ ]:
from BorutaShap import BorutaShap

In [ ]:
xgb_reg = xgb.XGBRegressor(random_state = 42)

Feature_Selector = BorutaShap(model = xgb_reg, 
                              importance_measure='shap', 
                              classification=False)
Feature_Selector.fit(X=X_train_ys, y= y_perm_train, n_trials=100, random_state=42)


In [ ]:
Feature_Selector.TentativeRoughFix()

In [ ]:
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
Feature_Selector.plot(X_size=10, figsize=(8,6),y_scale='log', which_features='all')

In [ ]:
subset = Feature_Selector.Subset()
subset.head()

In [ ]:
lgb_reg = lgb.LGBMRegressor(random_state = 42)

Feature_Selector_lgb = BorutaShap(model = lgb_reg, 
                              importance_measure='shap', 
                              classification=False)
Feature_Selector_lgb.fit(X=X_train_ys, y= y_perm_train, n_trials=100, random_state=42)

In [ ]:
Feature_Selector_lgb.TentativeRoughFix()

In [ ]:
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
Feature_Selector_lgb.plot(X_size=10, figsize=(8,6),y_scale='log', which_features='all')

In [ ]:
subset_lgb = Feature_Selector_lgb.Subset()
subset_lgb.head()

In [ ]:
rf_reg =  RandomForestRegressor(
        n_estimators = 50,  
        max_depth = 10,     
        random_state = 42)
Feature_Selector_rf = BorutaShap(model = rf_reg, 
                              importance_measure='shap', 
                              classification=False)
Feature_Selector_rf.fit(X=X_train_ys, y= y_perm_train, n_trials=100, random_state=42)

In [ ]:
Feature_Selector_rf.TentativeRoughFix()

In [ ]:
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
Feature_Selector_rf.plot(X_size=10, figsize=(8,6),y_scale='log', which_features='all')

In [ ]:
subset_rf = Feature_Selector_rf.Subset()
subset_rf.head()

In [ ]:
cat_reg =  cat.CatBoostRegressor(random_state = 42)
Feature_Selector_cat = BorutaShap(model = cat_reg , 
                              importance_measure='shap', 
                              classification=False)
Feature_Selector_cat.fit(X=X_train_ys, y= y_perm_train, n_trials=100, random_state=42)


In [ ]:
Feature_Selector_cat.TentativeRoughFix()

In [ ]:
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
Feature_Selector_cat.plot(X_size=10, figsize=(8,6),y_scale='log', which_features='all')

In [ ]:
subset_cat = Feature_Selector_cat.Subset()
subset_cat.head()

#    # Feature Subsets

In [ ]:
#original dataset
X_train_ys1 = X_train_ys
print(X_train_ys1)

In [ ]:
y_train_ys1 = y_train_ys
print(y_train_ys1)

In [ ]:
X_test_ys1 = X_test_ys
print(X_test_ys1)

In [ ]:
y_test_ys1 = y_test_ys
print(y_test_ys1)

In [ ]:
X_RFE_train_xg = X_train_ys.drop(['BIT','DTC','DTS'],axis=1)
X_RFE_train_xg.head()

In [ ]:
X_RFE_test_xg = X_test_ys.drop(['BIT','DTC','DTS'],axis=1)
X_RFE_test_xg.head()

In [ ]:
X_RFE_train_lg = X_train_ys.drop(['BIT'],axis=1)
X_RFE_train_lg.head()

In [ ]:
X_RFE_test_lg = X_test_ys.drop(['BIT'],axis=1)
X_RFE_test_lg.head()

In [ ]:
X_RFE_train_rf = X_train_ys.drop(['BIT'],axis=1)
X_RFE_train_rf.head()

In [ ]:
X_RFE_test_rf = X_test_ys.drop(['BIT'],axis=1)
X_RFE_test_rf.head()

In [ ]:
X_RFE_train_cat = X_train_ys.drop(['BIT','DTC','P28H'],axis=1)
X_RFE_train_cat.head()

In [ ]:
X_RFE_test_cat = X_test_ys.drop(['BIT','DTC','P28H'],axis=1)
X_RFE_test_cat.head()

In [ ]:
X_boruta_train_xg = X_train_ys.drop(['BIT','DTS','P28H'],axis=1)
X_boruta_train_xg.head()

In [ ]:
X_boruta_test_xg = X_test_ys.drop(['BIT','DTS','P28H'],axis=1)
X_boruta_test_xg.head()

In [ ]:
X_boruta_train_lg =  X_train_ys.drop(['BIT','DTS','P28H','DTC','P40H'],axis=1)
X_boruta_train_lg.head()

In [ ]:
X_boruta_test_lg = X_test_ys.drop(['BIT','DTS','P28H','DTC','P40H'],axis=1)
X_boruta_test_lg.head()

In [ ]:
X_boruta_train_rf = X_train_ys.drop(['BIT','DTS'],axis=1)
X_boruta_train_rf.head()

In [ ]:
X_boruta_test_rf = X_test_ys.drop(['BIT','DTS'],axis=1)
X_boruta_test_rf.head()

In [ ]:
X_boruta_train_cat = X_train_ys.drop(['CNL','P40H','P28H','BIT'],axis=1)
X_boruta_train_cat.head()

In [ ]:
X_boruta_test_cat = X_test_ys.drop(['CNL','P40H','P28H','BIT'],axis=1)
X_boruta_test_cat.head()

RF\XGBOOST\LIGHTGBM\NGBOOST

In [ ]:
#2 RANDOM FOREST 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_train_ys,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_train_ys)
y_perm_test_predict_rf = rf_reg.predict(X_test_ys)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_train_pearson,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_train_pearson)
y_perm_test_predict_rf = rf_reg.predict(X_test_pearson)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_RFE_train_xg,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_RFE_train_xg)
y_perm_test_predict_rf = rf_reg.predict(X_RFE_test_xg)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_RFE_train_lg,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_RFE_train_lg)
y_perm_test_predict_rf = rf_reg.predict(X_RFE_test_lg)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_RFE_train_rf,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_RFE_train_rf)
y_perm_test_predict_rf = rf_reg.predict(X_RFE_test_rf)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_RFE_train_cat,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_RFE_train_cat)
y_perm_test_predict_rf = rf_reg.predict(X_RFE_test_cat)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_boruta_train_xg,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_boruta_train_xg)
y_perm_test_predict_rf = rf_reg.predict(X_boruta_test_xg)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_boruta_train_lg,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_boruta_train_lg)
y_perm_test_predict_rf = rf_reg.predict(X_boruta_test_lg)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_boruta_train_rf,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_boruta_train_rf)
y_perm_test_predict_rf = rf_reg.predict(X_boruta_test_rf)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_rmse_test))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state =42) 
rf_reg.fit(X_boruta_train_cat,y_train_ys)
y_perm_train_predict_rf = rf_reg.predict(X_boruta_train_cat)
y_perm_test_predict_rf = rf_reg.predict(X_boruta_test_cat)
rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_rf)
rf_rmse_train = np.sqrt(rf_mse_train)
rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_rf)
rf_rmse_test = np.sqrt(rf_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_rmse_train))
rf_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_rf)
print('*  Train Score: %.4f MAE' % (rf_mae_train))
rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_rf)
print('*  Test Score: %.4f MAE' % (rf_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_rf))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_rf))

In [ ]:
#3 XGBoost

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_train_ys,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_train_ys)
y_perm_test_predict_xg = xg_reg.predict(X_test_ys)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_train_pearson,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_train_pearson)
y_perm_test_predict_xg = xg_reg.predict(X_test_pearson)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_RFE_train_xg,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_RFE_train_xg)
y_perm_test_predict_xg = xg_reg.predict(X_RFE_test_xg)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_RFE_train_lg,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_RFE_train_lg)
y_perm_test_predict_xg = xg_reg.predict(X_RFE_test_lg)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_RFE_train_rf,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_RFE_train_rf)
y_perm_test_predict_xg = xg_reg.predict(X_RFE_test_rf)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_RFE_train_cat,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_RFE_train_cat)
y_perm_test_predict_xg = xg_reg.predict(X_RFE_test_cat)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_boruta_train_xg,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_boruta_train_xg)
y_perm_test_predict_xg = xg_reg.predict(X_boruta_test_xg)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_boruta_train_lg,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_boruta_train_lg)
y_perm_test_predict_xg = xg_reg.predict(X_boruta_test_lg)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_boruta_train_rf,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_boruta_train_rf)
y_perm_test_predict_xg = xg_reg.predict(X_boruta_test_rf)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xg_reg =  xgb.XGBRegressor(random_state = 42)
xg_reg.fit(X_boruta_train_cat,y_train_ys)
y_perm_train_predict_xg = xg_reg.predict(X_boruta_train_cat)
y_perm_test_predict_xg = xg_reg.predict(X_boruta_test_cat)
xg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_xg)
xg_rmse_train = np.sqrt(xg_mse_train)
xg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_xg)
xg_rmse_test = np.sqrt(xg_mse_test)
print('*  Train Score: %.4f RMSE' % (xg_rmse_train))
print('*  Test Score: %.4f RMSE' % (xg_rmse_test))
xg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_xg)
print('*  Train Score: %.4f MAE' % (xg_mae_train))
xg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_xg)
print('*  Test Score: %.4f MAE' % (xg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_xg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_xg))

In [ ]:
#4 LightGBM

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_train_ys,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_train_ys)
y_perm_test_predict_lg = lg_reg.predict(X_test_ys)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_train_pearson,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_train_pearson)
y_perm_test_predict_lg = lg_reg.predict(X_test_pearson)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_RFE_train_xg,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_RFE_train_xg)
y_perm_test_predict_lg = lg_reg.predict(X_RFE_test_xg)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_RFE_train_lg,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_RFE_train_lg)
y_perm_test_predict_lg = lg_reg.predict(X_RFE_test_lg)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_RFE_train_rf,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_RFE_train_rf)
y_perm_test_predict_lg = lg_reg.predict(X_RFE_test_rf)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_RFE_train_cat,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_RFE_train_cat)
y_perm_test_predict_lg = lg_reg.predict(X_RFE_test_cat)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_boruta_train_xg,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_boruta_train_xg)
y_perm_test_predict_lg = lg_reg.predict(X_boruta_test_xg)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_boruta_train_lg,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_boruta_train_lg)
y_perm_test_predict_lg = lg_reg.predict(X_boruta_test_lg)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_boruta_train_rf,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_boruta_train_rf)
y_perm_test_predict_lg = lg_reg.predict(X_boruta_test_rf)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lg_reg = lgb.LGBMRegressor(random_state=42)
lg_reg.fit(X_boruta_train_cat,y_train_ys)
y_perm_train_predict_lg = lg_reg.predict(X_boruta_train_cat)
y_perm_test_predict_lg = lg_reg.predict(X_boruta_test_cat)
lg_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_lg)
lg_rmse_train = np.sqrt(lg_mse_train)
lg_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_lg)
lg_rmse_test = np.sqrt(lg_mse_test)
print('*  Train Score: %.4f RMSE' % (lg_rmse_train))
print('*  Test Score: %.4f RMSE' % (lg_rmse_test))
lg_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_lg)
print('*  Train Score: %.4f MAE' % (lg_mae_train))
lg_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_lg)
print('*  Test Score: %.4f MAE' % (lg_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_lg))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_lg))

In [ ]:
#5 NGBOOST

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_train_ys,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_train_ys)
y_perm_test_predict_ngb = ngb_reg.predict(X_test_ys)
ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))
ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_train_pearson,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_train_pearson)
y_perm_test_predict_ngb = ngb_reg.predict(X_test_pearson)
ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))
ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_RFE_train_xg,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_RFE_train_xg)
y_perm_test_predict_ngb = ngb_reg.predict(X_RFE_test_xg)
ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))
ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_RFE_train_lg,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_RFE_train_lg)
y_perm_test_predict_ngb = ngb_reg.predict(X_RFE_test_lg)
ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))
ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_RFE_train_rf,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_RFE_train_rf)
y_perm_test_predict_ngb = ngb_reg.predict(X_RFE_test_rf)
ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))
ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_RFE_train_cat,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_RFE_train_cat)
y_perm_test_predict_ngb = ngb_reg.predict(X_RFE_test_cat)
ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))
ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_boruta_train_xg,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_boruta_train_xg)
y_perm_test_predict_ngb = ngb_reg.predict(X_boruta_test_xg)

ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))

ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_boruta_train_lg,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_boruta_train_lg)
y_perm_test_predict_ngb = ngb_reg.predict(X_boruta_test_lg)

ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))

ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_boruta_train_rf,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_boruta_train_rf)
y_perm_test_predict_ngb = ngb_reg.predict(X_boruta_test_rf)

ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))

ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

In [ ]:
ngb_reg = NGBRegressor(random_state=42)
ngb_reg.fit(X_boruta_train_cat,y_train_ys)
y_perm_train_predict_ngb = ngb_reg.predict(X_boruta_train_cat)
y_perm_test_predict_ngb = ngb_reg.predict(X_boruta_test_cat)

ngb_mse_train = mean_squared_error(y_train_ys,y_perm_train_predict_ngb)
ngb_rmse_train = np.sqrt(ngb_mse_train)
ngb_mse_test = mean_squared_error(y_test_ys,y_perm_test_predict_ngb)
ngb_rmse_test = np.sqrt(ngb_mse_test)
print('*  Train Score: %.4f RMSE' % (ngb_rmse_train))
print('*  Test Score: %.4f RMSE' % (ngb_rmse_test))

ngb_mae_train = mean_absolute_error(y_train_ys, y_perm_train_predict_ngb)
print('*  Train Score: %.4f MAE' % (ngb_mae_train))
ngb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_predict_ngb)
print('*  Test Score: %.4f MAE' % (ngb_mae_test))
print("r2 score:", r2_score(y_train_ys, y_perm_train_predict_ngb))
print("r2 score:", r2_score(y_test_ys, y_perm_test_predict_ngb))

# Hyperparameter Optimization

In [ ]:
import datetime

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.font_manager import FontProperties


# Rondom Forest

# RF- PSO

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg_PSO = RandomForestRegressor(random_state =42)

# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid
    #numEstim_for = [200, 4 00, 500]
    #numEstim = np.arange(100,500,20)
    numEstim = int(solution[0])
    min_samples_split = int(solution[1])
    max_depth = int(solution[2])
    min_samples_leaf=int(solution[3])
    
    rf_reg_PSO = RandomForestRegressor(n_estimators = numEstim,min_samples_split = min_samples_split,max_depth = max_depth, min_samples_leaf=min_samples_leaf)
    # Fit the model
    rf_reg_PSO.fit(X_boruta_train_xg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_rf = rf_reg_PSO.predict(X_boruta_train_xg)
    y_perm_test_pred_rf = rf_reg_PSO.predict(X_boruta_test_xg)
    # Measure the performance
    #未进行反归一化
    
    #calculate root mean squared error均方根误差
    rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_rf)
    rf_rmse_train = np.sqrt(rf_mse_train)
    rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_rf)
    rf_rmse_test = np.sqrt(rf_mse_test)
    rf_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_rf)
    rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_rf)
    r2 = r2_score(y_test_ys,y_perm_test_pred_rf)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [100,    2,  2,2],
    "ub": [2000, 30, 20,30],
    "minmax": "max",
    "log_to": "file",
    "log_file": "rf_pso.log"
}
model_RF_PSO = PSO.BasePSO(problem, epoch=100, pop_size=50, c1=2.05, c2=2.05, w_min=0.4, w_max=0.9)
model_RF_PSO.solve()
best_position, best_fitness = model_RF_PSO.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_RF_PSO.history.list_global_best_fit, title='Global Best Fitness', filename='RF_PSO_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_RF_PSO.history.list_current_best_fit, title='Local Best Fitness', filename='RF_PSO_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_RF_PSO.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='RF_PSO_Runtime chart')        # Draw runtime for each generation
## On the exp loration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_RF_PSO.history.list_exploration, model_RF_PSO.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_RF_PSO.history.list_diversity], list_legends=['RF_PSO'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_RF_PSO.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='RF_PSO_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_RF_PSO.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='RF_PSO_Local Objectives Chart')

In [ ]:
rf_reg_PSO = RandomForestRegressor(n_estimators=100,min_samples_split=2, max_depth=20, min_samples_leaf=2,random_state=42)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
rf_reg_PSO.fit(X_boruta_train_xg,y_train_ys)

In [ ]:
y_perm_train_pred_rf_pso = rf_reg_PSO.predict(X_boruta_train_xg)
y_perm_test_pred_rf_pso = rf_reg_PSO.predict(X_boruta_test_xg)

In [ ]:
y_perm_train_pred_rf_pso = y_perm_train_pred_rf_pso.reshape(-1,1)
y_perm_test_pred_rf_pso = y_perm_test_pred_rf_pso.reshape(-1,1)

In [ ]:
rf_pso_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_rf_pso)
rf_pso_rmse_train = np.sqrt(rf_pso_mse_train)
rf_pso_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_rf_pso)
rf_pso_rmse_test = np.sqrt(rf_pso_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_pso_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_pso_rmse_test))
rf_pso_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_rf_pso)
print('*  Train Score: %.4f MAE' % (rf_pso_mae_train))
rf_pso_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_rf_pso)
print('*  Test Score: %.4f MAE' % (rf_pso_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_rf_pso))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_rf_pso))

In [ ]:
figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_rf_pso,s=30, c="r",marker='o',label='Train(R\u00b2=0.966)') 
plt.scatter(y_test_ys,y_perm_test_pred_rf_pso,s=30, c="b",marker='o',label='Test(R\u00b2=0.875)')  
#plt.grid()  
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'  
           , prop = font
           , markerscale = 2
           ,frameon = True
           ,framealpha=1
          ) 
#保存图片
plt.savefig('R2-RF-PSO.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# 4.4 PSO- GA

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg_GA = RandomForestRegressor(random_state =42)

# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid
    #numEstim_for = [200, 400, 500]
    #numEstim = np.arange(100,500,20)
    numEstim = int(solution[0])
    min_samples_split = int(solution[1])
    max_depth = int(solution[2])
    min_samples_leaf=int(solution[3])
    
    rf_reg_GA = RandomForestRegressor(n_estimators = numEstim,min_samples_split = min_samples_split,max_depth = max_depth, min_samples_leaf=min_samples_leaf)
    # Fit the model
    rf_reg_GA.fit(X_boruta_train_xg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_rf = rf_reg_GA.predict(X_boruta_train_xg)
    y_perm_test_pred_rf = rf_reg_GA.predict(X_boruta_test_xg)
    # Measure the performance
    #calculate root mean squared error
    rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_rf)
    rf_rmse_train = np.sqrt(rf_mse_train)
    rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_rf)
    rf_rmse_test = np.sqrt(rf_mse_test)
    rf_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_rf)
    rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_rf)
    r2 = r2_score(y_test_ys,y_perm_test_pred_rf)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [100,    2,  2,2],
    "ub": [2000, 30, 20,30],
    "minmax": "max",
}
model_RF_GA = GA.BaseGA(problem, epoch=100, pop_size=50, pc=0.95, pm=0.1, mutation_multipoints=True, mutation="flip")
model_RF_GA.solve()
best_position, best_fitness = model_RF_GA.solve()
print(f" Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_RF_GA.history.list_global_best_fit, title='Global Best Fitness', filename='RF_GA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_RF_GA.history.list_current_best_fit, title='Local Best Fitness', filename='RF_GA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_RF_GA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='RF_GA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_RF_GA.history.list_exploration, model_RF_GA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_RF_GA.history.list_diversity], list_legends=['RF_GA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_RF_GA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='RF_GA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_RF_GA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='RF_GA_Local Objectives Chart')

In [ ]:
rf_reg_GA = RandomForestRegressor(n_estimators=200,min_samples_split=4, max_depth=19, min_samples_leaf=3,random_state=42)
# Fit the model
rf_reg_GA.fit(X_boruta_train_xg,y_train_ys)

In [ ]:
y_perm_train_pred_rf_ga = rf_reg_GA.predict(X_boruta_train_xg)
y_perm_test_pred_rf_ga = rf_reg_GA.predict(X_boruta_test_xg)

In [ ]:
y_perm_train_pred_rf_ga = y_perm_train_pred_rf_ga.reshape(-1,1)
y_perm_test_pred_rf_ga = y_perm_test_pred_rf_ga.reshape(-1,1)

In [ ]:

rf_ga_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_rf_ga)
rf_ga_rmse_train = np.sqrt(rf_ga_mse_train)
rf_ga_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_rf_ga)
rf_ga_rmse_test = np.sqrt(rf_ga_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_ga_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_ga_rmse_test))
rf_ga_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_rf_ga)
print('*  Train Score: %.4f MAE' % (rf_ga_mae_train))
rf_ga_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_rf_ga)
print('*  Test Score: %.4f MAE' % (rf_ga_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_rf_ga))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_rf_ga))

In [ ]:
figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_rf_ga,s=30, c="r",marker='o',label='Train(R\u00b2=0.951)')  
plt.scatter(y_test_ys,y_perm_test_pred_rf_ga,s=30, c="b",marker='o',label='Test(R\u00b2=0.874)')  
#plt.grid()  
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   
           , prop = font
           , markerscale = 2
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-RF-GA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# 4.6 RF-SSA

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg_SSA = RandomForestRegressor(random_state =42)

# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid
    #numEstim_for = [200, 400, 500]
    #numEstim = np.arange(100,500,20)
    numEstim = int(solution[0])
    min_samples_split = int(solution[1])
    max_depth = int(solution[2])
    min_samples_leaf=int(solution[3])
    
    rf_reg_SSA = RandomForestRegressor(n_estimators = numEstim,min_samples_split = min_samples_split,max_depth = max_depth, min_samples_leaf=min_samples_leaf)
    # Fit the model
    rf_reg_SSA.fit(X_boruta_train_xg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_rf = rf_reg_SSA.predict(X_boruta_train_xg)
    y_perm_test_pred_rf = rf_reg_SSA.predict(X_boruta_test_xg)
    # Measure the performance
    #未进行反归一化
    #calculate root mean squared error均方根误差
    rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_rf)
    rf_rmse_train = np.sqrt(rf_mse_train)
    rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_rf)
    rf_rmse_test = np.sqrt(rf_mse_test)
    rf_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_rf)
    rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_rf)
    r2 = r2_score(y_test_ys,y_perm_test_pred_rf)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [100,    2,  2,2],
    "ub": [2000, 30, 20,30],
    "minmax": "max",
    "log_to": "file",
    "log_file": "rf_ssa.log"
}
model_RF_SSA = SSA.BaseSSA(problem, epoch=100, pop_size=50, ST=0.8, PD=0.2, SD=0.1)
model_RF_SSA.solve()
best_position, best_fitness = model_RF_SSA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_RF_SSA.history.list_global_best_fit, title='Global Best Fitness', filename='RF_SSA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_RF_SSA.history.list_current_best_fit, title='Local Best Fitness', filename='RF_SSA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_RF_SSA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='RF_SSA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_RF_SSA.history.list_exploration, model_RF_SSA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_RF_SSA.history.list_diversity], list_legends=['RF_SSA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_RF_SSA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='RF_SSA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_RF_SSA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='RF_SSA_Local Objectives Chart')

In [ ]:
rf_reg_SSA = RandomForestRegressor(n_estimators=1816,min_samples_split=2, max_depth=16, min_samples_leaf=2,random_state=42)
rf_reg_SSA.fit(X_boruta_train_xg,y_train_ys)

In [ ]:
y_perm_train_pred_rf_ssa = rf_reg_SSA.predict(X_boruta_train_xg)
y_perm_test_pred_rf_ssa = rf_reg_SSA.predict(X_boruta_test_xg)

In [ ]:
y_perm_train_pred_rf_ssa = y_perm_train_pred_rf_ssa.reshape(-1,1)
y_perm_test_pred_rf_ssa = y_perm_test_pred_rf_ssa.reshape(-1,1)

In [ ]:
rf_ssa_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_rf_ssa)
rf_ssa_rmse_train = np.sqrt(rf_ssa_mse_train)
rf_ssa_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_rf_ssa)
rf_ssa_rmse_test = np.sqrt(rf_ssa_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_ssa_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_ssa_rmse_test))
rf_ssa_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_rf_ssa)
print('*  Train Score: %.4f MAE' % (rf_ssa_mae_train))
rf_ssa_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_rf_ssa)
print('*  Test Score: %.4f MAE' % (rf_ssa_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_rf_ssa))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_rf_ssa))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_rf_ssa,s=30, c="r",marker='o',label='Train(R\u00b2=0.965)') 
plt.scatter(y_test_ys,y_perm_test_pred_rf_ssa,s=30, c="b",marker='o',label='Test(R\u00b2=0.875)')  
plt.xlim(-2, 3)
plt.ylim(-2, 3)
Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)
font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'  
           , prop = font
           , markerscale = 2#
           ,frameon = True
           ,framealpha=1
          ) 
plt.savefig('R2-RF-SSA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# RF-WOA

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg_WOA = RandomForestRegressor(random_state =42)

# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid
    #numEstim_for = [200, 400, 500]
    #numEstim = np.arange(100,500,20)
    numEstim = int(solution[0])
    min_samples_split = int(solution[1])
    max_depth = int(solution[2])
    min_samples_leaf=int(solution[3])
    
    rf_reg_WOA = RandomForestRegressor(n_estimators = numEstim,min_samples_split = min_samples_split,max_depth = max_depth, min_samples_leaf=min_samples_leaf)
    # Fit the model
    rf_reg_WOA.fit(X_boruta_train_xg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_rf = rf_reg_WOA.predict(X_boruta_train_xg)
    y_perm_test_pred_rf = rf_reg_WOA.predict(X_boruta_test_xg)
    # Measure the performance
    rf_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_rf)
    rf_rmse_train = np.sqrt(rf_mse_train)
    rf_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_rf)
    rf_rmse_test = np.sqrt(rf_mse_test)
    rf_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_rf)
    rf_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_rf)
    r2 = r2_score(y_test_ys,y_perm_test_pred_rf)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [100,    2,  2,2],
    "ub": [2000, 30, 20,30],
    "minmax": "max",
    "log_to": "file",
    "log_file": "rf_WOA.log"
}
model_RF_WOA = WOA.HI_WOA(problem, epoch=100, pop_size=50, feedback_max=5)
model_RF_WOA.solve()
best_position, best_fitness = model_RF_WOA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_RF_WOA.history.list_global_best_fit, title='Global Best Fitness', filename='RF_WOA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_RF_WOA.history.list_current_best_fit, title='Local Best Fitness', filename='RF_WOA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_RF_WOA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='RF_WOA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_RF_WOA.history.list_exploration, model_RF_WOA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_RF_WOA.history.list_diversity], list_legends=['RF_WOA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_RF_WOA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='RF_WOA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_RF_WOA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='RF_WOA_Local Objectives Chart')

In [ ]:
rf_reg_WOA = RandomForestRegressor(n_estimators=100,min_samples_split=2, max_depth=14, min_samples_leaf=2,random_state=42)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
rf_reg_WOA.fit(X_boruta_train_xg,y_train_ys)

In [ ]:
y_perm_train_pred_rf_woa = rf_reg_WOA.predict(X_boruta_train_xg)
y_perm_test_pred_rf_woa = rf_reg_WOA.predict(X_boruta_test_xg)

In [ ]:
y_perm_train_pred_rf_woa = y_perm_train_pred_rf_woa.reshape(-1,1)
y_perm_test_pred_rf_woa = y_perm_test_pred_rf_woa.reshape(-1,1)

In [ ]:
rf_woa_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_rf_woa)
rf_woa_rmse_train = np.sqrt(rf_woa_mse_train)
rf_woa_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_rf_woa)
rf_woa_rmse_test = np.sqrt(rf_woa_mse_test)
print('*  Train Score: %.4f RMSE' % (rf_woa_rmse_train))
print('*  Test Score: %.4f RMSE' % (rf_woa_rmse_test))
rf_woa_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_rf_woa)
print('*  Train Score: %.4f MAE' % (rf_woa_mae_train))
rf_woa_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_rf_woa)
print('*  Test Score: %.4f MAE' % (rf_woa_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_rf_woa))
print("Test R2 score:", r2_score(y_test_ys, y_perm_test_pred_rf_woa))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_rf_woa,s=30, c="r",marker='o',label='Train(R\u00b2=0.964)')  
plt.scatter(y_test_ys,y_perm_test_pred_rf_woa,s=30, c="b",marker='o',label='Test(R\u00b2=0.875)')  

plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'  
           , prop = font
           , markerscale = 2
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-RF-WOA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# 5 XGBOOST-PSO

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xgb_reg_PSO = xgb.XGBRegressor(random_state=42,tree_method='gpu_hist', gpu_id=0, n_jobs =-1)

In [ ]:
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid   
    
    learning_rate = solution[0]
    n_estimators = int(solution[1])
    max_depth = int(solution[2])
    min_child_weight = int(solution[3])
    subsample= solution[4]
    colsample_bytree = solution[5]
    
    xgb_reg_PSO = XGBRegressor(random_state=42,tree_method='gpu_hist', gpu_id=0, n_jobs =-1,colsample_bytree=colsample_bytree,learning_rate=learning_rate,max_depth=max_depth,n_estimators=n_estimators,min_child_weight=min_child_weight,subsample=subsample)
    # Fit the model
    xgb_reg_PSO.fit(X_RFE_train_xg, y_train_ys)
    # Make the predictions
    y_perm_train_pred_xgb = xgb_reg_PSO.predict(X_RFE_train_xg)
    y_perm_test_pred_xgb = xgb_reg_PSO.predict(X_RFE_test_xg)
    # Measure the performance
    xgb_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_xgb)
    xgb_rmse_train = np.sqrt(xgb_mse_train)
    xgb_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_xgb)
    xgb_rmse_test = np.sqrt(xgb_mse_test)
    xgb_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_xgb)
    xgb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_xgb)
    r2 = r2_score(y_test_ys,y_perm_test_pred_xgb)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01, 500,  1, 1,0.5,0.5],
    "ub": [0.3 ,1500, 11, 20,1,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "XG_PSO.log"
}
model_XGB_PSO = PSO.BasePSO(problem, epoch=100, pop_size=50, c1=2.05, c2=2.05, w_min=0.4, w_max=0.9)
model_XGB_PSO.solve()
best_position, best_fitness = model_XGB_PSO.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_XGB_PSO.history.list_global_best_fit, title='Global Best Fitness', filename='XGB_PSO_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_XGB_PSO.history.list_current_best_fit, title='Local Best Fitness', filename='XGB_PSO_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_XGB_PSO.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='XGB_PSO_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_XGB_PSO.history.list_exploration, model_XGB_PSO.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_XGB_PSO.history.list_diversity], list_legends=['XGB_PSO'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_XGB_PSO.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='XGB_PSO_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_XGB_PSO.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='XGB_PSO_Local Objectives Chart')

In [ ]:
xgb_reg_PSO = XGBRegressor(random_state=42,learning_rate =0.01,n_estimators =692,max_depth=11,min_child_weight=4,subsample=0.6,colsample_bytree=0.9)
xgb_reg_PSO.fit(X_RFE_train_xg, y_train_ys)

In [ ]:
y_perm_train_pred_xgb_pso = xgb_reg_PSO.predict(X_RFE_train_xg)
y_perm_test_pred_xgb_pso = xgb_reg_PSO.predict(X_RFE_test_xg)

In [ ]:
y_perm_train_pred_xgb_pso = y_perm_train_pred_xgb_pso.reshape(-1,1)
y_perm_test_pred_xgb_pso = y_perm_test_pred_xgb_pso.reshape(-1,1)

In [ ]:
xgb_pso_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_xgb_pso)
xgb_pso_rmse_train = np.sqrt(xgb_pso_mse_train)
xgb_pso_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_xgb_pso)
xgb_pso_rmse_test = np.sqrt(xgb_pso_mse_test)
print('*  Train Score: %.4f RMSE' % (xgb_pso_rmse_train))
print('*  Test Score: %.4f RMSE' % (xgb_pso_rmse_test))
xgb_pso_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_xgb_pso)
print('*  Train Score: %.4f MAE' % (xgb_pso_mae_train))
xgb_pso_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_xgb_pso)
print('*  Test Score: %.4f MAE' % (xgb_pso_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_xgb_pso))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_xgb_pso))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_xgb_pso,s=30, c="r",marker='o',label='Train(R\u00b2=0.986)')  
plt.scatter(y_test_ys,y_perm_test_pred_xgb_pso,s=30, c="b",marker='o',label='Test(R\u00b2=0.876)')  

plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 
plt.savefig('R2-XGB-PSO.jpg',dpi=1500, bbox_inches='tight')
plt.show()  #

# XGBOOST-GA

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xgb_reg_GA = xgb.XGBRegressor(random_state=42,tree_method='gpu_hist', gpu_id=0, n_jobs =-1)

In [ ]:
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid   
    
    learning_rate = solution[0]
    n_estimators = int(solution[1])
    max_depth = int(solution[2])
    min_child_weight = int(solution[3])
    subsample= solution[4]
    colsample_bytree = solution[5]
    
    xgb_reg_GA = XGBRegressor(colsample_bytree=colsample_bytree,learning_rate=learning_rate,max_depth=max_depth,n_estimators=n_estimators,min_child_weight=min_child_weight,subsample=subsample,random_state=42,tree_method='gpu_hist', gpu_id=0, n_jobs =-1)
    # Fit the model
    xgb_reg_GA.fit(X_RFE_train_xg, y_train_ys)
    # Make the predictions
    y_perm_train_pred_xgb = xgb_reg_GA.predict(X_RFE_train_xg)
    y_perm_test_pred_xgb = xgb_reg_GA.predict(X_RFE_test_xg)
    # Measure the performance
    xgb_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_xgb)
    xgb_rmse_train = np.sqrt(xgb_mse_train)
    xgb_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_xgb)
    xgb_rmse_test = np.sqrt(xgb_mse_test)
    xgb_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_xgb)
    xgb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_xgb)
    r2 = r2_score(y_test_ys,y_perm_test_pred_xgb)
    return r2
problem = {
    "fit_func": fitness_function,
    "lb": [0.01, 500,  1, 1,0.5,0.5],
    "ub": [0.3 ,1500, 11, 20,1,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "XG_GA.log"
}
model_XGB_GA = GA.BaseGA(problem, epoch=100, pop_size=50, pc=0.95, pm=0.1, mutation_multipoints=True, mutation="flip")
model_XGB_GA.solve()
best_position, best_fitness = model_XGB_GA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_XGB_GA.history.list_global_best_fit, title='Global Best Fitness', filename='XGB_GA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_XGB_GA.history.list_current_best_fit, title='Local Best Fitness', filename='XGB_GA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_XGB_GA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='XGB_GA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_XGB_GA.history.list_exploration, model_XGB_GA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_XGB_GA.history.list_diversity], list_legends=['XGB_GA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_XGB_GA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='XGB_GA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_XGB_GA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='XGB_GA_Local Objectives Chart')

In [ ]:
xgb_reg_GA = XGBRegressor(random_state=42,learning_rate =0.02,n_estimators =660,max_depth=7,min_child_weight=5,subsample=0.7,colsample_bytree=0.7)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
xgb_reg_GA.fit(X_RFE_train_xg, y_train_ys)

In [ ]:
y_perm_train_pred_xgb_ga = xgb_reg_GA.predict(X_RFE_train_xg)
y_perm_test_pred_xgb_ga = xgb_reg_GA.predict(X_RFE_test_xg)

In [ ]:
y_perm_train_pred_xgb_ga = y_perm_train_pred_xgb_ga.reshape(-1,1)
y_perm_test_pred_xgb_ga = y_perm_test_pred_xgb_ga.reshape(-1,1)

In [ ]:
xgb_ga_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_xgb_ga)
xgb_ga_rmse_train = np.sqrt(xgb_ga_mse_train)
xgb_ga_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_xgb_ga)
xgb_ga_rmse_test = np.sqrt(xgb_ga_mse_test)
print('*  Train Score: %.4f RMSE' % (xgb_ga_rmse_train))
print('*  Test Score: %.4f RMSE' % (xgb_ga_rmse_test))
xgb_ga_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_xgb_ga)
print('*  Train Score: %.4f MAE' % (xgb_ga_mae_train))
xgb_ga_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_xgb_ga)
print('*  Test Score: %.4f MAE' % (xgb_ga_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_xgb_ga))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_xgb_ga))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_xgb_ga,s=30, c="r",marker='o',label='Train(R\u00b2=0.981)')  
plt.scatter(y_test_ys,y_perm_test_pred_xgb_ga,s=30, c="b",marker='o',label='Test(R\u00b2=0.876)') 
#plt.grid()  
plt.xlim(-2, 3)
plt.ylim(-2, 3)
Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")
plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)
font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'  
           , prop = font
           , markerscale = 2
           ,frameon = True
           ,framealpha=1
          ) 
plt.savefig('R2-XGB-GA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# XGBOOST-SSA

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xgb_reg_SSA = xgb.XGBRegressor(random_state=42,tree_method='gpu_hist', gpu_id=0, n_jobs =-1)

In [ ]:
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid   
    
    learning_rate = solution[0]
    n_estimators = int(solution[1])
    max_depth = int(solution[2])
    min_child_weight = int(solution[3])
    subsample= solution[4]
    colsample_bytree = solution[5]
    
    xgb_reg_SSA = XGBRegressor(colsample_bytree=colsample_bytree,learning_rate=learning_rate,max_depth=max_depth,n_estimators=n_estimators,min_child_weight=min_child_weight,subsample=subsample,random_state=42,tree_method='gpu_hist', gpu_id=0, n_jobs =-1)
    # Fit the model
    xgb_reg_SSA.fit(X_RFE_train_xg, y_train_ys)
    # Make the predictions
    y_perm_train_pred_xgb = xgb_reg_SSA.predict(X_RFE_train_xg)
    y_perm_test_pred_xgb = xgb_reg_SSA.predict(X_RFE_test_xg)
  
    xgb_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_xgb)
    xgb_rmse_train = np.sqrt(xgb_mse_train)
    xgb_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_xgb)
    xgb_rmse_test = np.sqrt(xgb_mse_test)
    xgb_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_xgb)
    xgb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_xgb)
    r2 = r2_score(y_test_ys,y_perm_test_pred_xgb)
    return r2
problem = {
    "fit_func": fitness_function,
    "lb": [0.01, 500,  1, 1,0.5,0.5],
    "ub": [0.3 ,1500, 11, 20,1,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "XG_SSA.log"
}
model_XGB_SSA = SSA.BaseSSA(problem, epoch=100, pop_size=50, ST=0.8, PD=0.2, SD=0.1)
model_XGB_SSA.solve()
best_position, best_fitness = model_XGB_SSA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_XGB_SSA.history.list_global_best_fit, title='Global Best Fitness', filename='XGB_SSA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_XGB_SSA.history.list_current_best_fit, title='Local Best Fitness', filename='XGB_SSA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_XGB_SSA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='XGB_SSA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_XGB_SSA.history.list_exploration, model_XGB_SSA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_XGB_SSA.history.list_diversity], list_legends=['XGB_SSA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_XGB_SSA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='XGB_SSA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_XGB_SSA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='XGB_SSA_Local Objectives Chart')

In [ ]:
xgb_reg_SSA = XGBRegressor(random_state=42,learning_rate =0.01,n_estimators =658,max_depth=9,min_child_weight=1,subsample=0.7,colsample_bytree=0.6)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
xgb_reg_SSA.fit(X_RFE_train_xg, y_train_ys)

In [ ]:
y_perm_train_pred_xgb_ssa = xgb_reg_SSA.predict(X_RFE_train_xg)
y_perm_test_pred_xgb_ssa = xgb_reg_SSA.predict(X_RFE_test_xg)

In [ ]:
y_perm_train_pred_xgb_ssa = y_perm_train_pred_xgb_ssa.reshape(-1,1)
y_perm_test_pred_xgb_ssa = y_perm_test_pred_xgb_ssa.reshape(-1,1)

In [ ]:

xgb_ssa_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_xgb_ssa)
xgb_ssa_rmse_train = np.sqrt(xgb_ssa_mse_train)
xgb_ssa_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_xgb_ssa)
xgb_ssa_rmse_test = np.sqrt(xgb_ssa_mse_test)
print('*  Train Score: %.4f RMSE' % (xgb_ssa_rmse_train))
print('*  Test Score: %.4f RMSE' % (xgb_ssa_rmse_test))

xgb_ssa_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_xgb_ssa)
print('*  Train Score: %.4f MAE' % (xgb_ssa_mae_train))
xgb_ssa_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_xgb_ssa)
print('*  Test Score: %.4f MAE' % (xgb_ssa_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_xgb_ssa))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_xgb_ssa))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_xgb_ssa,s=30, c="r",marker='o',label='Train(R\u00b2=0.989)') 
plt.scatter(y_test_ys,y_perm_test_pred_xgb_ssa,s=30, c="b",marker='o',label='Test(R\u00b2=0.876)')  
#plt.grid()  
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-XGB-SSA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# XGBOOST -WOA

In [ ]:
from xgboost import plot_importance
import xgboost as xgb
from xgboost import XGBRegressor
xgb_reg_WOA = xgb.XGBRegressor(random_state=42,tree_method='gpu_hist', gpu_id=0, n_jobs =-1)

In [ ]:
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid   
    
    learning_rate = solution[0]
    n_estimators = int(solution[1])
    max_depth = int(solution[2])
    min_child_weight = int(solution[3])
    subsample= solution[4]
    colsample_bytree = solution[5]
    
    xgb_reg_WOA = XGBRegressor(colsample_bytree=colsample_bytree,learning_rate=learning_rate,max_depth=max_depth,n_estimators=n_estimators,min_child_weight=min_child_weight,subsample=subsample,random_state=42,tree_method='gpu_hist', gpu_id=0, n_jobs =-1)
    # Fit the model
    xgb_reg_WOA.fit(X_RFE_train_xg, y_train_ys)
    # Make the predictions
    y_perm_train_pred_xgb = xgb_reg_WOA.predict(X_RFE_train_xg)
    y_perm_test_pred_xgb = xgb_reg_WOA.predict(X_RFE_test_xg)
    # Measure the performance

    xgb_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_xgb)
    xgb_rmse_train = np.sqrt(xgb_mse_train)
    xgb_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_xgb)
    xgb_rmse_test = np.sqrt(xgb_mse_test)
    xgb_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_xgb)
    xgb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_xgb)
    r2 = r2_score(y_test_ys,y_perm_test_pred_xgb)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01, 500,  1, 1,0.5,0.5],
    "ub": [0.3 ,1500, 11, 20,1,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "XG_WOA.log"
}
model_XGB_WOA =WOA.HI_WOA(problem, epoch=100, pop_size=50, feedback_max=5)
model_XGB_WOA.solve()
best_position, best_fitness = model_XGB_WOA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_XGB_WOA.history.list_global_best_fit, title='Global Best Fitness', filename='XGB_WOA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_XGB_WOA.history.list_current_best_fit, title='Local Best Fitness', filename='XGB_WOA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_XGB_WOA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='XGB_WOA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_XGB_WOA.history.list_exploration, model_XGB_WOA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_XGB_WOA.history.list_diversity], list_legends=['XGB_WOA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_XGB_WOA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='XGB_WOA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_XGB_WOA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='XGB_WOA_Local Objectives Chart')

In [ ]:
xgb_reg_WOA = XGBRegressor(random_state=42,learning_rate =0.02,n_estimators =597,max_depth=7,min_child_weight=15,subsample=0.7,colsample_bytree=0.5)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
xgb_reg_WOA.fit(X_RFE_train_xg, y_train_ys)

In [ ]:
y_perm_train_pred_xgb_woa = xgb_reg_WOA.predict(X_RFE_train_xg)
y_perm_test_pred_xgb_woa = xgb_reg_WOA.predict(X_RFE_test_xg)

In [ ]:
y_perm_train_pred_xgb_woa = y_perm_train_pred_xgb_woa.reshape(-1,1)
y_perm_test_pred_xgb_woa = y_perm_test_pred_xgb_woa.reshape(-1,1)

In [ ]:

xgb_woa_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_xgb_woa)
xgb_woa_rmse_train = np.sqrt(xgb_woa_mse_train)
xgb_woa_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_xgb_woa)
xgb_woa_rmse_test = np.sqrt(xgb_woa_mse_test)
print('*  Train Score: %.4f RMSE' % (xgb_woa_rmse_train))
print('*  Test Score: %.4f RMSE' % (xgb_woa_rmse_test))

xgb_woa_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_xgb_woa)
print('*  Train Score: %.4f MAE' % (xgb_woa_mae_train))
xgb_woa_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_xgb_woa)
print('*  Test Score: %.4f MAE' % (xgb_woa_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_xgb_woa))
print("Test R2 score:", r2_score(y_test_ys, y_perm_test_pred_xgb_woa))

In [ ]:
#
figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_xgb_ga,s=30, c="r",marker='o',label='Train(R\u00b2=0.955)')  #
plt.scatter(y_test_ys,y_perm_test_pred_xgb_ga,s=30, c="b",marker='o',label='Test(R\u00b2=0.872)')  
#plt.grid()  
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'  
           , prop = font
           , markerscale = 2
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-XGB-WOA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# 6 lightGBM模型

# LightGBM -PSO

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lgb_reg_PSO = lgb.LGBMRegressor(random_state=42,device = 'gpu')

In [ ]:
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid   
    learning_rate_lgb = solution[0]
    n_estimators = int(solution[1])
    max_depth_lgb = int(solution[2])
    feature_fraction_lgb = solution[3]
    bagging_fraction_lgb = solution[4]
    
    lgb_reg_PSO = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=learning_rate_lgb,max_depth=max_depth_lgb,feature_fraction=feature_fraction_lgb,bagging_fraction=bagging_fraction_lgb,n_estimators =n_estimators)
    # Fit the model
    lgb_reg_PSO.fit(X_boruta_train_lg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_lgb = lgb_reg_PSO.predict(X_boruta_train_lg)
    y_perm_test_pred_lgb = lgb_reg_PSO.predict(X_boruta_test_lg)
    # Measure the performance
    #未进行反归一化
    #calculate root mean squared error均方根误差
    lgb_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_lgb)
    lgb_rmse_train = np.sqrt(lgb_mse_train)
    lgb_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_lgb)
    lgb_rmse_test = np.sqrt(lgb_mse_test)
    lgb_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_lgb)
    lgb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_lgb)
    r2 = r2_score(y_test_ys, y_perm_test_pred_lgb)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01, 100,  1, 0.5,0.5],
    "ub": [0.3 , 3000,11, 1,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "LG_PSO.log"
}
model_LGB_PSO = PSO.BasePSO(problem, epoch=100, pop_size=50, c1=2.05, c2=2.05, w_min=0.4, w_max=0.9)
model_LGB_PSO.solve()
best_position, best_fitness = model_LGB_PSO.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_LGB_PSO.history.list_global_best_fit, title='Global Best Fitness', filename='LGB_PSO_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_LGB_PSO.history.list_current_best_fit, title='Local Best Fitness', filename='LGB_PSO_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_LGB_PSO.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='LGB_PSO_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_LGB_PSO.history.list_exploration, model_LGB_PSO.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_LGB_PSO.history.list_diversity], list_legends=['LGB_PSO'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_LGB_PSO.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='LGB_PSO_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_LGB_PSO.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='LGB_PSO_Local Objectives Chart')

In [ ]:
lgb_reg_PSO = LGBMRegressor(random_state=42,learning_rate=0.13,n_estimators =100,max_depth=11,feature_fraction =1,bagging_fraction=0.5)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
lgb_reg_PSO.fit(X_boruta_train_lg,y_train_ys)

In [ ]:
y_perm_train_pred_lgb_pso = lgb_reg_PSO.predict(X_boruta_train_lg)
y_perm_test_pred_lgb_pso = lgb_reg_PSO.predict(X_boruta_test_lg)

In [ ]:
y_perm_train_pred_lgb_pso = y_perm_train_pred_lgb_pso.reshape(-1,1)
y_perm_test_pred_lgb_pso = y_perm_test_pred_lgb_pso.reshape(-1,1)

In [ ]:

lgb_pso_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_lgb_pso)
lgb_pso_rmse_train = np.sqrt(lgb_pso_mse_train)
lgb_pso_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_lgb_pso)
lgb_pso_rmse_test = np.sqrt(lgb_pso_mse_test)
print('*  Train Score: %.4f RMSE' % (lgb_pso_rmse_train))
print('*  Test Score: %.4f RMSE' % (lgb_pso_rmse_test))

lgb_pso_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_lgb_pso)
print('*  Train Score: %.4f MAE' % (lgb_pso_mae_train))
lgb_pso_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_lgb_pso)
print('*  Test Score: %.4f MAE' % (lgb_pso_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_lgb_pso))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_lgb_pso))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_lgb_pso,s=30, c="r",marker='o',label='Train(R\u00b2=0.961)')  #
plt.scatter(y_test_ys,y_perm_test_pred_lgb_pso,s=30, c="b",marker='o',label='Test(R\u00b2=0.862)')  #

plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   #
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-LGB-PSO.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# LightGBM-GA

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lgb_reg_GA = lgb.LGBMRegressor(random_state=42,device = 'gpu')

In [ ]:
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid   
    learning_rate_lgb = solution[0]
    n_estimators = int(solution[1])
    max_depth_lgb = int(solution[2])
    feature_fraction_lgb = solution[3]
    bagging_fraction_lgb = solution[4]
    
    lgb_reg_GA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=learning_rate_lgb,max_depth=max_depth_lgb,feature_fraction=feature_fraction_lgb,bagging_fraction=bagging_fraction_lgb,n_estimators =n_estimators)
    # Fit the model
    lgb_reg_GA.fit(X_boruta_train_lg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_lgb = lgb_reg_GA.predict(X_boruta_train_lg)
    y_perm_test_pred_lgb = lgb_reg_GA.predict(X_boruta_test_lg)
    # Measure the performance
    #未进行反归一化
    #calculate root mean squared error均方根误差
    lgb_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_lgb)
    lgb_rmse_train = np.sqrt(lgb_mse_train)
    lgb_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_lgb)
    lgb_rmse_test = np.sqrt(lgb_mse_test)
    lgb_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_lgb)
    lgb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_lgb)
    r2 = r2_score(y_test_ys, y_perm_test_pred_lgb)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01, 100,  1, 0.5,0.5],
    "ub": [0.3 , 3000,11, 1,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "LG_GA.log"
}
model_LGB_GA = GA.BaseGA(problem, epoch=100, pop_size=50, pc=0.95, pm=0.1, mutation_multipoints=True, mutation="flip")
model_LGB_GA.solve()
best_position, best_fitness = model_LGB_GA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_LGB_GA.history.list_global_best_fit, title='Global Best Fitness', filename='LGB_GA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_LGB_GA.history.list_current_best_fit, title='Local Best Fitness', filename='LGB_GA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_LGB_GA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='LGB_GA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_LGB_GA.history.list_exploration, model_LGB_GA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_LGB_GA.history.list_diversity], list_legends=['LGB_GA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_LGB_GA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='LGB_GA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_LGB_GA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='LGB_GA_Local Objectives Chart')

In [ ]:
lgb_reg_GA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=0.09,n_estimators =131,max_depth=10,feature_fraction =0.7,bagging_fraction=0.8)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
lgb_reg_GA.fit(X_boruta_train_lg,y_train_ys)

In [ ]:
y_perm_train_pred_lgb_ga = lgb_reg_GA.predict(X_boruta_train_lg)
y_perm_test_pred_lgb_ga = lgb_reg_GA.predict(X_boruta_test_lg)

In [ ]:
y_perm_train_pred_lgb_ga = y_perm_train_pred_lgb_ga.reshape(-1,1)
y_perm_test_pred_lgb_ga = y_perm_test_pred_lgb_ga.reshape(-1,1)

In [ ]:

lgb_ga_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_lgb_ga)
lgb_ga_rmse_train = np.sqrt(lgb_ga_mse_train)
lgb_ga_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_lgb_ga)
lgb_ga_rmse_test = np.sqrt(lgb_ga_mse_test)
print('*  Train Score: %.4f RMSE' % (lgb_ga_rmse_train))
print('*  Test Score: %.4f RMSE' % (lgb_ga_rmse_test))

lgb_ga_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_lgb_ga)
print('*  Train Score: %.4f MAE' % (lgb_ga_mae_train))
lgb_ga_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_lgb_ga)
print('*  Test Score: %.4f MAE' % (lgb_ga_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_lgb_ga))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_lgb_ga))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_lgb_pso,s=30, c="r",marker='o',label='Train(R\u00b2=0.951)')  
plt.scatter(y_test_ys,y_perm_test_pred_lgb_pso,s=30, c="b",marker='o',label='Test(R\u00b2=0.870)')  
#plt.grid()  # 
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-LGB-GA.jpg',dpi=1500, bbox_inches='tight')
plt.show() 

# LightGBM-SSA

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lgb_reg_SSA = lgb.LGBMRegressor(random_state=42,device = 'gpu')

In [ ]:
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid   
    learning_rate_lgb = solution[0]
    n_estimators = int(solution[1])
    max_depth_lgb = int(solution[2])
    feature_fraction_lgb = solution[3]
    bagging_fraction_lgb = solution[4]
    
    lgb_reg_SSA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=learning_rate_lgb,max_depth=max_depth_lgb,feature_fraction=feature_fraction_lgb,bagging_fraction=bagging_fraction_lgb,n_estimators =n_estimators)
    # Fit the model
    lgb_reg_SSA.fit(X_boruta_train_lg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_lgb = lgb_reg_SSA.predict(X_boruta_train_lg)
    y_perm_test_pred_lgb = lgb_reg_SSA.predict(X_boruta_test_lg)
    # Measure the performance
    lgb_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_lgb)
    lgb_rmse_train = np.sqrt(lgb_mse_train)
    lgb_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_lgb)
    lgb_rmse_test = np.sqrt(lgb_mse_test)
    lgb_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_lgb)
    lgb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_lgb)
    r2 = r2_score(y_test_ys, y_perm_test_pred_lgb)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01, 100,  1, 0.5,0.5],
    "ub": [0.3 , 3000,11, 1,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "XG_SSA.log"
}
model_LGB_SSA = SSA.BaseSSA(problem, epoch=100, pop_size=50, ST=0.8, PD=0.2, SD=0.1)
model_LGB_SSA.solve()
best_position, best_fitness = model_LGB_SSA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_LGB_SSA.history.list_global_best_fit, title='Global Best Fitness', filename='LGB_SSA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_LGB_SSA.history.list_current_best_fit, title='Local Best Fitness', filename='LGB_SSA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_LGB_SSA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='LGB_SSA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_LGB_SSA.history.list_exploration, model_LGB_SSA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_LGB_SSA.history.list_diversity], list_legends=['LGB_SSA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_LGB_SSA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='LGB_SSA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_LGB_SSA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='LGB_SSA_Local Objectives Chart')

In [ ]:
lgb_reg_SSA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=0.03,n_estimators =368,max_depth=11,feature_fraction =1,bagging_fraction=0.8)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
lgb_reg_SSA.fit(X_boruta_train_lg,y_train_ys)

In [ ]:
y_perm_train_pred_lgb_ssa = lgb_reg_SSA.predict(X_boruta_train_lg)
y_perm_test_pred_lgb_ssa = lgb_reg_SSA.predict(X_boruta_test_lg)

In [ ]:
y_perm_train_pred_lgb_ssa = y_perm_train_pred_lgb_ssa.reshape(-1,1)
y_perm_test_pred_lgb_ssa = y_perm_test_pred_lgb_ssa.reshape(-1,1)

In [ ]:

lgb_ssa_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_lgb_ssa)
lgb_ssa_rmse_train = np.sqrt(lgb_ssa_mse_train)
lgb_ssa_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_lgb_ssa)
lgb_ssa_rmse_test = np.sqrt(lgb_ssa_mse_test)
print('*  Train Score: %.4f RMSE' % (lgb_ssa_rmse_train))
print('*  Test Score: %.4f RMSE' % (lgb_ssa_rmse_test))

lgb_ssa_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_lgb_ssa)
print('*  Train Score: %.4f MAE' % (lgb_ssa_mae_train))
lgb_ssa_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_lgb_ssa)
print('*  Test Score: %.4f MAE' % (lgb_ssa_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_lgb_ssa))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_lgb_ssa))

In [ ]:
#
figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_lgb_ssa,s=30, c="r",marker='o',label='Train(R\u00b2=0.954)')  
plt.scatter(y_test_ys,y_perm_test_pred_lgb_ssa,s=30, c="b",marker='o',label='Test(R\u00b2=0.869)')  
#plt.grid()  # 
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   #
           , prop = font
           , markerscale = 2#
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-LGB-SSA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# LightGBM -WOA

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance
import lightgbm as lgb
lgb_reg_WOA = lgb.LGBMRegressor(random_state=42,device = 'gpu')

In [ ]:
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid   
    learning_rate_lgb = solution[0]
    n_estimators = int(solution[1])
    max_depth_lgb = int(solution[2])
    feature_fraction_lgb = solution[3]
    bagging_fraction_lgb = solution[4]
    
    lgb_reg_WOA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=learning_rate_lgb,max_depth=max_depth_lgb,feature_fraction=feature_fraction_lgb,bagging_fraction=bagging_fraction_lgb,n_estimators =n_estimators)
    # Fit the model
    lgb_reg_WOA.fit(X_boruta_train_lg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_lgb = lgb_reg_WOA.predict(X_boruta_train_lg)
    y_perm_test_pred_lgb = lgb_reg_WOA.predict(X_boruta_test_lg)
    # Measure the performance

    lgb_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_lgb)
    lgb_rmse_train = np.sqrt(lgb_mse_train)
    lgb_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_lgb)
    lgb_rmse_test = np.sqrt(lgb_mse_test)
    lgb_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_lgb)
    lgb_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_lgb)
    r2 = r2_score(y_test_ys, y_perm_test_pred_lgb)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01, 100,  1, 0.5,0.5],
    "ub": [0.3 , 3000,11, 1,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "LG_WOA.log"
}
model_LGB_WOA = WOA.HI_WOA(problem, epoch=100, pop_size=50, feedback_max=5)
model_LGB_WOA.solve()
best_position, best_fitness = model_LGB_WOA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_LGB_WOA.history.list_global_best_fit, title='Global Best Fitness', filename='LGB_WOA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_LGB_WOA.history.list_current_best_fit, title='Local Best Fitness', filename='LGB_WOA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_LGB_WOA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='LGB_WOA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_LGB_WOA.history.list_exploration, model_LGB_WOA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_LGB_WOA.history.list_diversity], list_legends=['LGB_WOA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_LGB_WOA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='LGB_WOA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_LGB_WOA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='LGB_WOA_Local Objectives Chart')

In [ ]:
lgb_reg_WOA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=0.17,n_estimators =100,max_depth=6,feature_fraction =0.7,bagging_fraction=0.9)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
lgb_reg_WOA.fit(X_boruta_train_lg,y_train_ys)

In [ ]:
y_perm_train_pred_lgb_woa = lgb_reg_WOA.predict(X_boruta_train_lg)
y_perm_test_pred_lgb_woa = lgb_reg_WOA.predict(X_boruta_test_lg)

In [ ]:
y_perm_train_pred_lgb_woa = y_perm_train_pred_lgb_woa.reshape(-1,1)
y_perm_test_pred_lgb_woa = y_perm_test_pred_lgb_woa.reshape(-1,1)

In [ ]:

lgb_woa_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_lgb_woa)
lgb_woa_rmse_train = np.sqrt(lgb_woa_mse_train)
lgb_woa_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_lgb_woa)
lgb_woa_rmse_test = np.sqrt(lgb_woa_mse_test)
print('*  Train Score: %.4f RMSE' % (lgb_woa_rmse_train))
print('*  Test Score: %.4f RMSE' % (lgb_woa_rmse_test))

lgb_woa_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_lgb_woa)
print('*  Train Score: %.4f MAE' % (lgb_woa_mae_train))
lgb_woa_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_lgb_woa)
print('*  Test Score: %.4f MAE' % (lgb_woa_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_lgb_woa))
print("Test R2 score:", r2_score(y_test_ys, y_perm_test_pred_lgb_woa))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_lgb_woa,s=30, c="r",marker='o',label='Train(R\u00b2=0.948)')  #
plt.scatter(y_test_ys,y_perm_test_pred_lgb_woa,s=30, c="b",marker='o',label='Test(R\u00b2=0.867)')  
#plt.grid()  # 
plt.xlim(-2, 3)
plt.ylim(-2, 3)
#
Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   # 
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 
#保存图片
plt.savefig('R2-LGB-WOA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  # 显示图片

# # NGBoost: Natural Gradient Boosting for Probabilistic Prediction 

#  NGBOOST- PSO

In [ ]:
ng_reg_PSO = NGBRegressor(random_state=42,natural_gradient=True)

In [ ]:
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid
    learning_rate = solution[0]
    n_estimators = int(solution[1])
    minibatch_frac = solution[2]
    ng_reg_PSO = NGBRegressor(learning_rate=learning_rate,n_estimators=n_estimators,minibatch_frac=minibatch_frac )
    # Fit the model
    ng_reg_PSO.fit(X_RFE_train_xg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_ng = ng_reg_PSO.predict(X_RFE_train_xg)
    y_perm_test_pred_ng = ng_reg_PSO.predict(X_RFE_test_xg)
    # Measure the performance

    ng_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_ng)
    ng_rmse_train = np.sqrt(ng_mse_train)
    ng_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_ng)
    ng_rmse_test = np.sqrt(ng_mse_test)
    ng_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_ng)
    ng_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_ng)
    r2 = r2_score(y_test_ys,y_perm_test_pred_ng)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01,100,0.5],
    "ub": [0.1,1500,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "NG_PSO.log"
}
model_NGB_PSO = PSO.BasePSO(problem, epoch=100, pop_size=50, c1=2.05, c2=2.05, w_min=0.4, w_max=0.9)
model_NGB_PSO.solve()
best_position, best_fitness = model_NGB_PSO.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_NGB_PSO.history.list_global_best_fit, title='Global Best Fitness', filename='NGB_PSO_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_NGB_PSO.history.list_current_best_fit, title='Local Best Fitness', filename='NGB_PSO_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_NGB_PSO.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='NGB_PSO_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_NGB_PSO.history.list_exploration, model_NGB_PSO.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_NGB_PSO.history.list_diversity], list_legends=['NGB_PSO'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_NGB_PSO.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='NGB_PSO_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_NGB_PSO.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='NGB_PSO_Local Objectives Chart')

In [ ]:
ng_reg_PSO = NGBRegressor(learning_rate=0.08,n_estimators=1500,minibatch_frac=0.8,random_state=42,natural_gradient=True)
# Fit the model
ng_reg_PSO.fit(X_RFE_train_xg,y_train_ys)

In [ ]:
y_perm_train_pred_ng_pso = ng_reg_PSO.predict(X_RFE_train_xg)
y_perm_test_pred_ng_pso = ng_reg_PSO.predict(X_RFE_test_xg)

In [ ]:
y_perm_train_pred_ng_pso = y_perm_train_pred_ng_pso.reshape(-1,1)
y_perm_test_pred_ng_pso = y_perm_test_pred_ng_pso.reshape(-1,1)

In [ ]:
#
ng_pso_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_ng_pso)
ng_pso_rmse_train = np.sqrt(ng_pso_mse_train)
ng_pso_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_ng_pso)
ng_pso_rmse_test = np.sqrt(ng_pso_mse_test)
print('*  Train Score: %.4f RMSE' % (ng_pso_rmse_train))
print('*  Test Score: %.4f RMSE' % (ng_pso_rmse_test))

ng_pso_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_ng_pso)
print('*  Train Score: %.4f MAE' % (ng_pso_mae_train))
ng_pso_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_ng_pso)
print('*  Test Score: %.4f MAE' % (ng_pso_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_ng_pso))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_ng_pso))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_ng_pso,s=30, c="r",marker='o',label='Train(R\u00b2=0.989)')  
plt.scatter(y_test_ys,y_perm_test_pred_ng_pso,s=30, c="b",marker='o',label='Test(R\u00b2=0.859)')  
#plt.grid()  
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)
#
font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   #
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-NGB-PSO.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

#  NGBOOST-GA

In [ ]:
ng_reg_GA = NGBRegressor(random_state=42,natural_gradient=True)

In [ ]:
# LABEL ENCODER
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid
    learning_rate = solution[0]
    n_estimators = int(solution[1])
    minibatch_frac = solution[2]
    ng_reg_GA = NGBRegressor(learning_rate=learning_rate,n_estimators=n_estimators,minibatch_frac=minibatch_frac,random_state=42,natural_gradient=True )
    # Fit the model
    ng_reg_GA.fit(X_RFE_train_xg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_ng = ng_reg_GA.predict(X_RFE_train_xg)
    y_perm_test_pred_ng = ng_reg_GA.predict(X_RFE_test_xg)
    # Measure the performance

    ng_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_ng)
    ng_rmse_train = np.sqrt(ng_mse_train)
    ng_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_ng)
    ng_rmse_test = np.sqrt(ng_mse_test)
    ng_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_ng)
    ng_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_ng)
    r2 = r2_score(y_test_ys,y_perm_test_pred_ng)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01,100,0.5],
    "ub": [0.1,1500,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "NG_GA.log"
}
model_NGB_GA = GA.BaseGA(problem, epoch=100, pop_size=50, pc=0.95, pm=0.1, mutation_multipoints=True, mutation="flip")
model_NGB_GA.solve()
best_position, best_fitness = model_NGB_GA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_NGB_GA.history.list_global_best_fit, title='Global Best Fitness', filename='NGB_GA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_NGB_GA.history.list_current_best_fit, title='Local Best Fitness', filename='NGB_GA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_NGB_GA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='NGB_GA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_NGB_GA.history.list_exploration, model_NGB_GA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_NGB_GA.history.list_diversity], list_legends=['NGB_GA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_NGB_GA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='NGB_GA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_NGB_GA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='NGB_GA_Local Objectives Chart')

In [ ]:
ng_reg_GA = NGBRegressor(learning_rate=0.03,n_estimators=1473,minibatch_frac=0.7,random_state=42,natural_gradient=True)
# Fit the model
ng_reg_GA.fit(X_RFE_train_xg,y_train_ys)

In [ ]:
y_perm_train_pred_ng_ga = ng_reg_GA.predict(X_RFE_train_xg)
y_perm_test_pred_ng_ga = ng_reg_GA.predict(X_RFE_test_xg)

In [ ]:
y_perm_train_pred_ng_ga = y_perm_train_pred_ng_ga.reshape(-1,1)
y_perm_test_pred_ng_ga = y_perm_test_pred_ng_ga.reshape(-1,1)

In [ ]:

ng_ga_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_ng_ga)
ng_ga_rmse_train = np.sqrt(ng_ga_mse_train)
ng_ga_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_ng_ga)
ng_ga_rmse_test = np.sqrt(ng_ga_mse_test)
print('*  Train Score: %.4f RMSE' % (ng_ga_rmse_train))
print('*  Test Score: %.4f RMSE' % (ng_ga_rmse_test))

ng_ga_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_ng_ga)
print('*  Train Score: %.4f MAE' % (ng_ga_mae_train))
ng_ga_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_ng_ga)
print('*  Test Score: %.4f MAE' % (ng_ga_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_ng_ga))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_ng_ga))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_ng_ga,s=30, c="r",marker='o',label='Train(R\u00b2=0.963)')  
plt.scatter(y_test_ys,y_perm_test_pred_ng_ga,s=30, c="b",marker='o',label='Test(R\u00b2=0.866)')  #
#plt.grid()  # 
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   #
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 
#保存图片
plt.savefig('R2-NGB-GA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  # 显示图片

# NGBOOST-SSA

In [ ]:
ng_reg_SSA = NGBRegressor(random_state=42,natural_gradient=True)

In [ ]:
# LABEL ENCODER
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid
    learning_rate = solution[0]
    n_estimators = int(solution[1])
    minibatch_frac = solution[2]
    ng_reg_SSA = NGBRegressor(learning_rate=learning_rate,n_estimators=n_estimators,minibatch_frac=minibatch_frac )
    ng_reg_SSA.fit(X_RFE_train_xg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_ng = ng_reg_SSA.predict(X_RFE_train_xg)
    y_perm_test_pred_ng = ng_reg_SSA.predict(X_RFE_test_xg)
    # Measure the performance
    ng_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_ng)
    ng_rmse_train = np.sqrt(ng_mse_train)
    ng_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_ng)
    ng_rmse_test = np.sqrt(ng_mse_test)
    ng_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_ng)
    ng_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_ng)
    r2 = r2_score(y_test_ys,y_perm_test_pred_ng)
    return r2

problem = {
    "fit_func": fitness_function,
    "lb": [0.01,100,0.5],
    "ub": [0.1,1500,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "NG_SSA.log"
}
model_NGB_SSA =SSA.BaseSSA(problem, epoch=100, pop_size=50, ST=0.8, PD=0.2, SD=0.1)

model_NGB_SSA.solve()
best_position, best_fitness = model_NGB_SSA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_NGB_SSA.history.list_global_best_fit, title='Global Best Fitness', filename='NGB_SSA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_NGB_SSA.history.list_current_best_fit, title='Local Best Fitness', filename='NGB_SSA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_NGB_SSA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='NGB_SSA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_NGB_SSA.history.list_exploration, model_NGB_SSA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_NGB_SSA.history.list_diversity], list_legends=['NGB_SSA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_NGB_SSA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='NGB_SSA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_NGB_SSA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='NGB_SSA_Local Objectives Chart')

In [ ]:
ng_reg_SSA = NGBRegressor(learning_rate=0.08,n_estimators=1500,minibatch_frac=0.8,random_state=42,natural_gradient=True)
# Fit the model
ng_reg_SSA.fit(X_RFE_train_xg,y_train_ys)

In [ ]:
y_perm_train_pred_ng_ssa = ng_reg_SSA.predict(X_RFE_train_xg)
y_perm_test_pred_ng_ssa = ng_reg_SSA.predict(X_RFE_test_xg)

In [ ]:
y_perm_train_pred_ng_ssa = y_perm_train_pred_ng_ssa.reshape(-1,1)
y_perm_test_pred_ng_ssa = y_perm_test_pred_ng_ssa.reshape(-1,1)

In [ ]:

ng_ssa_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_ng_ssa)
ng_ssa_rmse_train = np.sqrt(ng_ssa_mse_train)
ng_ssa_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_ng_ssa)
ng_ssa_rmse_test = np.sqrt(ng_ssa_mse_test)
print('*  Train Score: %.4f RMSE' % (ng_ssa_rmse_train))
print('*  Test Score: %.4f RMSE' % (ng_ssa_rmse_test))

ng_ssa_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_ng_ssa)
print('*  Train Score: %.4f MAE' % (ng_ssa_mae_train))
ng_ssa_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_ng_ssa)
print('*  Test Score: %.4f MAE' % (ng_ssa_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_ng_ssa))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_ng_ssa))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_ng_ssa,s=30, c="r",marker='o',label='Train(R\u00b2=0.989)')  
plt.scatter(y_test_ys,y_perm_test_pred_ng_ssa,s=30, c="b",marker='o',label='Test(R\u00b2=0.860)')  # 
#plt.grid()  # 
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 
#保存图片
plt.savefig('R2-NGB-SSA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  #

# NGBOOST-WOA

In [ ]:
ng_reg_WOA = NGBRegressor(random_state=42,natural_gradient=True)

In [ ]:
# LABEL ENCODER
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))
starttime = datetime.datetime.now()
def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid
    learning_rate = solution[0]
    n_estimators = int(solution[1])
    minibatch_frac = solution[2]
    ng_reg_WOA = NGBRegressor(learning_rate=learning_rate,n_estimators=n_estimators,minibatch_frac=minibatch_frac,random_state=42,natural_gradient=True )
    # Fit the model
    ng_reg_WOA.fit(X_RFE_train_xg,y_train_ys)
    # Make the predictions
    y_perm_train_pred_ng = ng_reg_WOA.predict(X_RFE_train_xg)
    y_perm_test_pred_ng = ng_reg_WOA.predict(X_RFE_test_xg)
    # Measure the performance
    ng_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_ng)
    ng_rmse_train = np.sqrt(ng_mse_train)
    ng_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_ng)
    ng_rmse_test = np.sqrt(ng_mse_test)
    ng_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_ng)
    ng_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_ng)
    r2 = r2_score(y_test_ys,y_perm_test_pred_ng)
    return r2

problem = {
    
    "fit_func": fitness_function,
    "lb": [0.01,100,0.5],
    "ub": [0.1,1500,1],
    "minmax": "max",
    "log_to": "file",
    "log_file": "NG_WOA.log"
}
model_NGB_WOA = WOA.HI_WOA(problem, epoch=100, pop_size=50, feedback_max=5)
model_NGB_WOA.solve()
best_position, best_fitness = model_NGB_WOA.solve()
print(f"Best solution: {best_position}, Best fitness: {best_fitness}")
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
export_convergence_chart(model_NGB_WOA.history.list_global_best_fit, title='Global Best Fitness', filename='NGB_WOA_Global Best Fitness')            # Draw global best fitness found so far in previous generations
export_convergence_chart(model_NGB_WOA.history.list_current_best_fit, title='Local Best Fitness', filename='NGB_WOA_Local Best Fitness')             # Draw current best fitness in each previous generation
export_convergence_chart(model_NGB_WOA.history.list_epoch_time, title='Runtime chart', y_label="Second", filename='NGB_WOA_Runtime chart')        # Draw runtime for each generation
## On the exploration and exploitation in popular swarm-based metaheuristic algorithms

# This exploration/exploitation chart should draws for single algorithm and single fitness function
export_explore_exploit_chart([model_NGB_WOA.history.list_exploration, model_NGB_WOA.history.list_exploitation])  # Draw exploration and exploitation chart

# This diversity chart should draws for multiple algorithms for a single fitness function at the same time to compare the diversity spreading
export_diversity_chart([model_NGB_WOA.history.list_diversity], list_legends=['NGB_WOA'])        # Draw diversity measurement chart

## Because convergence chart is formulated from objective values and weights, thus we also want to draw objective charts to understand the convergence
# Need a little bit more pre-processing
global_obj_list = np.array([agent[1][1] for agent in model_NGB_WOA.history.list_global_best])     # 2D array / matrix 2D
global_obj_list = [global_obj_list[:,idx] for idx in range(0, len(global_obj_list[0]))]     # Make each obj_list as a element in array for drawing
export_objectives_chart(global_obj_list, title='Global Objectives Chart', filename='NGB_WOA_Global Objectives Chart')
current_obj_list = np.array([agent[1][1] for agent in model_NGB_WOA.history.list_current_best])  # 2D array / matrix 2D
current_obj_list = [current_obj_list[:, idx] for idx in range(0, len(current_obj_list[0]))]  # Make each obj_list as a element in array for drawing
export_objectives_chart(current_obj_list, title='Local Objectives Chart', filename='NGB_WOA_Local Objectives Chart')

In [ ]:
ng_reg_WOA = NGBRegressor(learning_rate=0.0477398081,n_estimators=1486,minibatch_frac=0.688610492,random_state=42,natural_gradient=True)
# Fit the model
ng_reg_WOA.fit(X_RFE_train_xg,y_train_ys)

In [ ]:
y_perm_train_pred_ng_woa = ng_reg_WOA.predict(X_RFE_train_xg)
y_perm_test_pred_ng_woa = ng_reg_WOA.predict(X_RFE_test_xg)

In [ ]:
y_perm_train_pred_ng_woa = y_perm_train_pred_ng_woa.reshape(-1,1)
y_perm_test_pred_ng_woa = y_perm_test_pred_ng_woa.reshape(-1,1)

In [ ]:

ng_woa_mse_train = mean_squared_error(y_train_ys,y_perm_train_pred_ng_woa)
ng_woa_rmse_train = np.sqrt(ng_woa_mse_train)
ng_woa_mse_test = mean_squared_error(y_test_ys,y_perm_test_pred_ng_woa)
ng_woa_rmse_test = np.sqrt(ng_woa_mse_test)
print('*  Train Score: %.4f RMSE' % (ng_woa_rmse_train))
print('*  Test Score: %.4f RMSE' % (ng_woa_rmse_test))
ng_woa_mae_train = mean_absolute_error(y_train_ys,y_perm_train_pred_ng_woa)
print('*  Train Score: %.4f MAE' % (ng_woa_mae_train))
ng_woa_mae_test = mean_absolute_error(y_test_ys, y_perm_test_pred_ng_woa)
print('*  Test Score: %.4f MAE' % (ng_woa_mae_test))

In [ ]:
print("Train R2 score:", r2_score(y_train_ys,y_perm_train_pred_ng_woa))
print("Test R2 score:", r2_score(y_test_ys,y_perm_test_pred_ng_woa))

In [ ]:

figsize = 4, 4
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(y_train_ys,y_perm_train_pred_ng_woa,s=30, c="r",marker='o',label='Train(R\u00b2=0.977)')  #
plt.scatter(y_test_ys,y_perm_test_pred_ng_woa,s=30, c="b",marker='o',label='Test(R\u00b2=0.862)')  #
#plt.grid()  #
plt.xlim(-2, 3)
plt.ylim(-2, 3)

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=3,color='black',label="1:1 Line")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured Log K(mD)",fontproperties="Times New Roman",fontsize=20)
plt.ylabel("Predicted Log K(mD)", fontproperties="Times New Roman",fontsize=20)

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'bold'  # 'normal' 
#         ,'color':'red'
        ,'size':8
       }
plt.legend(loc = 'upper left'   
           , prop = font
           , markerscale = 2# 
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('R2-NGB-WOA.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# Blind test

In [ ]:
lgb_reg_WOA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=0.17,n_estimators =100,max_depth=6,feature_fraction =0.7,bagging_fraction=0.9)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
lgb_reg_WOA.fit(X_boruta_train_lg,y_train_ys)

In [ ]:
val_set = pd.read_csv('BlindTestWellA.csv')
val_set.head()

In [ ]:
val_set = val_set.values
val_set = val_set.astype('float32')

In [ ]:
X_val = val_set
print(X_val)

In [ ]:
X_val_pd=X_val

In [ ]:
y_perm_val_predict_xgb = lgb_reg_WOA.predict(X_val_pd)
print(y_perm_val_predict_xgb.shape)

In [ ]:
y_perm_val_predict_xgb = y_perm_val_predict_xgb.reshape(-1,1)
#inverse_xgb_perm_val = min_max_scaler_y_perm.inverse_transform(y_perm_val_predict_xgb)

In [ ]:
print(y_perm_val_predict_xgb.shape)

In [ ]:
xgb_perm_val=pd.DataFrame(10**y_perm_val_predict_xgb)
xgb_perm_val.to_csv('vali_xgb.csv')

In [ ]:
val_blindtest = pd.read_csv('Validataion.csv')
val_blindtest.head()

In [ ]:
val_blindtest = val_blindtest.values
val_blindtest = val_blindtest.astype('float32')

In [ ]:
perm_core = val_blindtest[:,0]
perm_fit = val_blindtest[:,1]
perm_predict = val_blindtest[:,2]

In [ ]:

mse_Core_Fit = mean_squared_error(perm_core,perm_fit)
rmse_Core_Fit = np.sqrt(mse_Core_Fit)
mse_Core_Predict = mean_squared_error(perm_core,perm_predict)
rmse_Core_Predict = np.sqrt(mse_Core_Predict)
print('*  Core_Fit: %.4f RMSE' % (rmse_Core_Fit))
print('*  Core_Predict: %.4f RMSE' % (rmse_Core_Predict))

mae_Core_Fit = mean_absolute_error(perm_core,perm_fit)
print('*  Core_Fit: %.4f MAE' % (mae_Core_Fit))
mae_Core_Predict = mean_absolute_error(perm_core,perm_predict)
print('*  Core_Predict: %.4f MAE' % (mae_Core_Predict))

In [ ]:
Core_Fitr2 = r2_score(perm_core,perm_fit)
Core_Predictr2 = r2_score(perm_core,perm_predict)
print("Core_Fitr2 score:", r2_score(perm_core,perm_fit))
print("Core_Predictr2 score:", r2_score(perm_core,perm_predict))

In [ ]:
lgb_reg_WOA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=0.17,n_estimators =100,max_depth=6,feature_fraction =0.7,bagging_fraction=0.9)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)
# Fit the model
lgb_reg_WOA.fit(X_boruta_train_lg,y_train_ys)

In [ ]:
val_set = pd.read_csv('BlindTestWellA.csv')
val_set.head()

In [ ]:
val_set = val_set.values
val_set = val_set.astype('float32')

In [ ]:
X_val = val_set
print(X_val)

In [ ]:
X_val_pd=X_val

In [ ]:
y_perm_val_predict_xgb = lgb_reg_WOA.predict(X_val_pd)
print(y_perm_val_predict_xgb.shape)

In [ ]:
y_perm_val_predict_xgb = y_perm_val_predict_xgb.reshape(-1,1)
#inverse_xgb_perm_val = min_max_scaler_y_perm.inverse_transform(y_perm_val_predict_xgb)

In [ ]:
print(y_perm_val_predict_xgb.shape)

In [ ]:
xgb_perm_val=pd.DataFrame(10**y_perm_val_predict_xgb)
xgb_perm_val.to_csv('vali_xgb.csv')

In [ ]:
val_blindtest = pd.read_csv('WELLAVALIDATION.csv')
val_blindtest.head()

In [ ]:
val_blindtest = val_blindtest.values
val_blindtest = val_blindtest.astype('float32')

In [ ]:
perm_core = val_blindtest[:,0]
perm_fit = val_blindtest[:,1]
fk_fit = val_blindtest[:,2]
perm_predict = val_blindtest[:,3]

In [ ]:

mse_Core_Fit = mean_squared_error(perm_core,perm_fit)
rmse_Core_Fit = np.sqrt(mse_Core_Fit)
mse_Core_FK = mean_squared_error(perm_core,fk_fit)
rmse_Core_FK = np.sqrt(mse_Core_FK)
mse_Core_Predict = mean_squared_error(perm_core,perm_predict)
rmse_Core_Predict = np.sqrt(mse_Core_Predict)
print('*  Core_Fit: %.4f RMSE' % (rmse_Core_Fit))
print('*  Core_FK: %.4f RMSE' % (rmse_Core_FK))
print('*  Core_Predict: %.4f RMSE' % (rmse_Core_Predict))

mae_Core_Fit = mean_absolute_error(perm_core,perm_fit)
print('*  Core_Fit: %.4f MAE' % (mae_Core_Fit))
mae_Core_FK = mean_absolute_error(perm_core,fk_fit)
print('*  Core_FK: %.4f MAE' % (mae_Core_FK))
mae_Core_Predict = mean_absolute_error(perm_core,perm_predict)
print('*  Core_Predict: %.4f MAE' % (mae_Core_Predict))

In [ ]:
Core_Fitr2 = r2_score(perm_core,perm_fit)
Core_Predictr2 = r2_score(perm_core,perm_predict)
print("Core_Fitr2 score:", r2_score(perm_core,perm_fit))
print("Core_FK score:", r2_score(perm_core,fk_fit))
print("Core_Predictr2 score:", r2_score(perm_core,perm_predict))

In [ ]:

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
figsize = 6, 6
figure, ax = plt.subplots(figsize=figsize)
plt.scatter(perm_core,perm_fit,s=80, c="r",marker='o',label='Por-K Model(R\u00b2=0.182)',edgecolors='black',linewidths=1,alpha=1) 
plt.scatter(perm_core,fk_fit,s=80, c="fuchsia",marker='o',label='F-K Model(R\u00b2=-0.070)',edgecolors='black',linewidths=1,alpha=1)  
plt.scatter(perm_core,perm_predict,s=80, c="b",marker='o',label='LGB-WOA(R\u00b2=0.797)',edgecolors='black',linewidths=1,alpha=1)  
#plt.grid()  #
plt.xlim(0.01,1000)
plt.ylim(0.01,1000)
plt.yscale('log')#
plt.xscale('log')#

Axis_line=np.linspace(*ax.get_xlim(),2)
ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle='--',linewidth=2,color='black',label="Y=X")

plt.tick_params(labelsize=15)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]

plt.xlabel("Measured K(mD)",fontproperties="Times New Roman",fontsize=20,weight='normal')
plt.ylabel("Predicted K(mD)",fontproperties="Times New Roman",fontsize=20,weight='normal')

font = {'family':'Times New Roman'  #'serif', 
#         ,'style':'italic'
        ,'weight':'normal'  # 'normal' 
#         ,'color':'red'
        ,'size':13
       }
plt.legend(loc = 'upper left'   
           , prop = font
           , markerscale = 1# 
           ,frameon = True
           ,framealpha=1
          ) 

plt.savefig('BlindTest17-1-2.jpg',dpi=1500, bbox_inches='tight')
plt.show()  

# shap

In [ ]:
lgb_reg_WOA = LGBMRegressor(random_state=42,device = 'gpu',learning_rate=0.17,n_estimators =100,max_depth=6,feature_fraction =0.7,bagging_fraction=0.9)
#rf_reg = SVC(C=c, random_state=1, kernel=kernel_decoded)

In [ ]:
import shap
import matplotlib as mpl
shap.initjs()

In [ ]:
model = lgb_reg_WOA.fit(X_boruta_train_lg,y_train_ys)

In [ ]:
type(y_train_ys)

In [ ]:
print(np.argmax(y_train_ys))

In [ ]:
print(np.max(y_train_ys))

In [ ]:
print(np.median(y_train_ys))

In [ ]:
a = y_train_ys[y_train_ys.PERM==0.16731733].index.tolist()
print(a)

In [ ]:
print(np.argmin(y_train_ys))

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(X_boruta_train_lg)
expected_value = explainer.expected_value

In [ ]:
# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
# visualize the first prediction's explanation
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
shap.plots.waterfall(shap_values[184],max_display=11,show=False)

mpl.rcParams['text.color'] = 'black'



fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') # 
    text.set_color('black') # 
    text.set_fontsize(15)
plt.savefig('waterfall-median.jpg', dpi=600, bbox_inches='tight')

In [ ]:
# summarize the effects of all the features
   
config = {
    "font.family": "Times New Roman",  #
    "font.serif": ["Times New Roman"],  # 
    "font.size": 20,  # 
    "axes.unicode_minus": False,
    "mathtext.fontset": "stix",  # 
}
plt.rcParams.update(config)
plt.rcParams['font.sans-serif'] = [u'SimHei'] 
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['xtick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.beeswarm(shap_values,max_display=11,show=False)
plt.xticks( fontproperties='Times New Roman', size=20) 
plt.yticks(fontproperties='Times New Roman', size=20) #
plt.xlabel('SHAP value', fontsize=20)#
plt.tight_layout() 

#
mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') # 
    text.set_color('black') # 
    text.set_fontsize(20)
plt.savefig('beeswarm1.jpg', dpi=1500, bbox_inches='tight')

In [ ]:

config = {
    "font.family": "Times New Roman",  # 
    "font.serif": ["Times New Roman"],  #
    "font.size": 20,  
    "axes.unicode_minus": False,
    "mathtext.fontset": "stix",  #
}
plt.rcParams.update(config)
plt.rcParams['font.sans-serif'] = [u'SimHei'] 
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['xtick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.summary_plot(shap_values,X_boruta_train_lg, plot_type="bar",max_display=11,show=False)
plt.xlabel("Mean(|Shap Value|)",fontproperties="Times New Roman",fontsize=20)
plt.xticks( fontproperties='Times New Roman', size=20) 
plt.yticks(fontproperties='Times New Roman', size=20) #


mpl.rcParams['text.color'] = 'black'

#
fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal')
    text.set_color('black') # 
plt.tight_layout() #
plt.savefig('feature importance_bar.jpg', dpi=500, bbox_inches='tight')

# scatter

In [ ]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.scatter(shap_values[:,"POR_LOG"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) 
plt.yticks(fontproperties='Times New Roman', size=20) 
plt.xlabel('POR_LOG', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') 
    text.set_color('black') #
plt.tight_layout() 
plt.savefig('SCATTER-POR.jpg', dpi=500, bbox_inches='tight')

In [ ]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.scatter(shap_values[:,"SH"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) 
plt.yticks(fontproperties='Times New Roman', size=20) 
plt.xlabel('SH', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') #
    text.set_color('black') #
plt.tight_layout() #
plt.savefig('SCATTER-SH.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.scatter(shap_values[:,"KJ"],show=False)
plt.xticks( fontproperties='Times New Roman', size=15) #
plt.yticks(fontproperties='Times New Roman', size=15) 

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('bold') 
    text.set_color('black') #
plt.tight_layout() 
plt.savefig('SCATTER-KJ.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.scatter(shap_values[:,"P16H"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) 
plt.yticks(fontproperties='Times New Roman', size=20) #
plt.xlabel('P16H', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') 
    text.set_color('black') # 
plt.tight_layout() 
plt.savefig('SCATTER-P16H.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.scatter(shap_values[:,"GR"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) 
plt.yticks(fontproperties='Times New Roman', size=20) #
plt.xlabel('POR_LOG', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)#
#
mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') 
    text.set_color('black') #
plt.tight_layout() #
plt.savefig('SCATTER-GR.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.scatter(shap_values[:,"DEN"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) #
plt.yticks(fontproperties='Times New Roman', size=20) 
plt.xlabel('DEN', fontsize=20)
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') #
    text.set_color('black') # 
plt.tight_layout() #
plt.savefig('SCATTER-DEN.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.scatter(shap_values[:,"CAL"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) 
plt.yticks(fontproperties='Times New Roman', size=20) 
plt.xlabel('CAL', fontsize=20)
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()
#
for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') # 
    text.set_color('black') # 
plt.tight_layout() #
plt.savefig('SCATTER-CAL.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
#plt.rcParams['font.sans-serif'] = ['Times New Roman']
#plt.rcParams['font.family'] = ['sans-serif']
shap.plots.scatter(shap_values[:,"CNL"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) #
plt.yticks(fontproperties='Times New Roman', size=20) #
plt.xlabel('CNL', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') # 
    text.set_color('black') # 
plt.tight_layout() #
plt.savefig('SCATTER-CNL.jpg', dpi=1500, bbox_inches='tight')

# Simple dependence plot ¶

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
config = {
    "font.size": 20,  # 五号，10.5磅
    "axes.unicode_minus": False,
}
plt.rcParams.update(config)
plt.rcParams.update(config)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
shap.plots.scatter(shap_values[:,"POR_LOG"], color=shap_values[:,"KJ"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) #
plt.yticks(fontproperties='Times New Roman', size=20) 
plt.xlabel('POR_LOG', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') 
    text.set_color('black') #
    text.set_fontsize(20)
plt.tight_layout() #
plt.savefig('KJ-POR.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
shap.plots.scatter(shap_values[:,"POR_LOG"], color=shap_values[:,"P16H"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) #
plt.yticks(fontproperties='Times New Roman', size=20) #
plt.xlabel('POR_LOG', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') 
    text.set_color('black') # 
    text.set_fontsize(20)
plt.tight_layout() #
plt.savefig('P16H-POR.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
shap.plots.scatter(shap_values[:,"POR_LOG"], color=shap_values[:,"DEN"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) #
plt.yticks(fontproperties='Times New Roman', size=20) 
plt.xlabel('POR_LOG', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'


fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') 
    text.set_color('black') #
    text.set_fontsize(20)
plt.tight_layout() 
plt.savefig('DEN-POR.jpg', dpi=1500, bbox_inches='tight')

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
shap.plots.scatter(shap_values[:,"POR_LOG"], color=shap_values[:,"CAL"],show=False)
plt.xticks( fontproperties='Times New Roman', size=20) 
plt.yticks(fontproperties='Times New Roman', size=20) 
plt.xlabel('POR_LOG', fontsize=20)#
plt.ylabel('SHAP Value', fontsize=20)#

mpl.rcParams['text.color'] = 'black'

#
fig = plt.gcf()

for text in fig.findobj(plt.Text):
    text.set_fontweight('normal') # 
    text.set_color('black') #
    text.set_fontsize(20)
plt.tight_layout() #
plt.savefig('CAL-POR.jpg', dpi=1500, bbox_inches='tight')